In [ ]:
# %%

import pandas as pd

from source.utils import FREQ_DIR, RESULT_DIR, UCS_DIR, confirm_dir
from source.utils.associate import (BINARY_ASSOC_ARGS, add_extra_am,
                                    associate_ucs, confirm_basic_ucs)
from source.utils.associate import convert_ucs_to_csv as ucs2csv
from source.utils.associate import get_associations_csv as init_am, AM_DF_DIR
from source.utils.associate import manipulate_ucs, seek_readable_ucs, adjust_assoc_columns
pd.set_option('display.float_format', '{:,.2f}'.format)

# %% [markdown]

 set parameters:
 - `UNIT` options:
   - `'Bigr'`
   - `'Adv'`
   - `'Adj'`
 - `PAT_DIR` options:
   - any directory in `results/freq_out/` containing a `ucs_format/` subdir with a `Trig*.tsv` frequency table
   - developed with the following in mind:
     - `POSmirror`
     - `NEGmirror`
     - `ANYmirror` (which has been populated by simple concatenation in the shell)
 UNIT = 'Adv'

In [ ]:
UNIT = 'Bigr' 
# UNIT = 'Adv'
# PAT_DIR = 'POSmirror'
# PAT_DIR = 'NEGmirror'
PAT_DIR = 'ANYmirror'
# FRQ_FLOOR = 3
FRQ_FLOOR = 10
# FRQ_FLOOR = 20
# FRQ_FLOOR = 50
# FRQ_FLOOR = 100
TRIG_TSV = FREQ_DIR.joinpath(
    f'{PAT_DIR}/ucs_format/Trig{UNIT}_frq-thrMIN-7.35f.tsv')
FOCUS = ['f', 'unexpected_f',
         'conservative_log_ratio',
         'am_p1_given2', 'am_p2_given1',
         'am_p1_given2_simple', 'am_p2_given1_simple',
         'am_log_likelihood',
        #  'mutual_information', 'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11',
         'l1', 'l2']

In [ ]:
# %%

TRIGGER_POLARITY = {
    'positive': {'all',
                 'always',
                 'both',
                 'either',
                 'every',
                 'everybody',
                 'everyone',
                 'everything',
                 'many',
                 'often',
                 'or',
                 'some',
                 'somebody',
                 'someone',
                 'something',
                 'somethings',
                 'sometimes'},
    'negative': {'barely',
                 'hardly',
                 'neither',
                 'never',
                 'no',
                 'nobody',
                 'none',
                 'nor',
                 'nothing',
                 'seldom',
                 'rarely',
                 'scarcely'},

}

In [ ]:
# %%

TRIGGER_QUANT = {
    'existential': {
        'some',
        'somebody',
        'someone',
        'something',
        'somethings',
        'either',
     'or',
        'sometimes'
    },
    'universal': {
        'all', 'every',
        'always',
        'everybody',
        'everyone',
        'both',
        'everything',
    },
    'not_exist': {
        'neither',
        'never',
        'no',
        'nobody',
        'none',
        'nor',
        'nothing',
    },
    'hedged_not_exist': {
        'barely',
        'hardly',
        'rarely',
        'scarcely',
        'seldom',
        'few'
    },
    'hedged_universal':{
     'many',
     'often',
     }
}

In [ ]:
# %%

def invert_set_dict(d: dict):
    return {v: k for k in d for v in d[k]}

# %% [markdown]

## 1. Run `seek_readable_ucs()` to generate consistent output path

In [ ]:
readable = seek_readable_ucs(min_freq=FRQ_FLOOR,
                             ucs_subdir='trigger_eval',
                             contained_counts_path=TRIG_TSV)
print(readable.relative_to(RESULT_DIR))

    > seeking `trigger_eval/ANYmirror/readable/TrigBigr_frq-thrMIN-7.35f_min10x*` frequency data and initial associations...
ucs/trigger_eval/ANYmirror/readable/TrigBigr_frq-thrMIN-7.35f_min10x.rsort-view_am-only.txt


# %% [markdown]

 Snippet of starting frequency data (`TRIG_TSV`)

In [ ]:
! head -5 {TRIG_TSV} | column -t

12233  something  as_simple
7372   something  completely_different
6933   all        too_familiar
6505   all        too_easy
5141   or         more_likely


# %% [markdown]

## 2. Run `confirm_basic_ucs()` (if needed)

In [ ]:
if not readable.is_file():
    basic_ucs_path = readable.parent.parent.joinpath(
        readable.name.replace('.rsort-view_am-only.txt', '.ds.gz'))
    print(f'Creating initial UCS table: `{basic_ucs_path.relative_to(RESULT_DIR)}')

    basic_ucs_path = confirm_basic_ucs(
        basic_ucs_path,
        freq_floor=FRQ_FLOOR,
        contained_counts_path=TRIG_TSV)

Creating initial UCS table: `ucs/trigger_eval/ANYmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz

## Creating initial UCS table...

```
( cat /share/compling/projects/sanpi/results/freq_out/ANYmirror/ucs_format/TrigBigr_frq-thrMIN-7.35f.tsv \ 
  | ucs-make-tables --types --threshold=10 /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz ) \ 
  && ucs-sort /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz BY f2- f1- INTO /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz
== Note ==
    N = total number of tokens/all counts summed
    V = total number of rows/number of unique l1+l2 combinations before filtering to 10+ tokens
+ time to make table → 00:00:02.201
```

Saving initial frequency table in readable .txt format...

```
ucs-print -o /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/readable/TrigBigr_frq-thrM

# %% [markdown]

 Excerpt of initial UCS table

In [ ]:
init_readable = UCS_DIR.joinpath(
    f'trigger_eval/{PAT_DIR}/readable'
).joinpath(f'{TRIG_TSV.name.replace(".tsv","")}_min{FRQ_FLOOR}x.init.txt')
! head -7 {init_readable}

        l1  l2                                f     f2      f1        N  
----------  ----------------------------  -----  -----  ------  -------  
        or  more_important                 1067  23085  326614  1761853  
 something  more_important                 2970  23085  318343  1761853  
       all  more_important                 3987  23085  249279  1761853  
      some  more_important                  321  23085  114929  1761853  
     never  more_important                 3853  23085  109723  1761853  


# %% [markdown]

## 3. Run `associate_ucs()` (if needed)

In [ ]:
if not readable.is_file():
    associate_ucs(basic_ucs_path)

transform_ucs_log = f'/share/compling/projects/sanpi/logs/associate/ucs//ucs-{PAT_DIR}_Trig{UNIT}_frq-thrMIN-7-35f_min{FRQ_FLOOR}x*.log'
! head -15 `ls -t1 {transform_ucs_log} | head -1`
! echo '...'
! tail -2 `ls -t1 {transform_ucs_log} | head -1`


Calculating UCS associations...

```
bash /share/compling/projects/sanpi/script/transform_ucs.sh /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz
> log will be saved to: /share/compling/projects/sanpi/logs/associate/ucs//ucs-ANYmirror_TrigBigr_frq-thrMIN-7-35f_min10x.2024-05-25_2135.log
...
```

+ time elapsed → 00:00:23.61
# Manipulating TrigBigr_frq-thrMIN-7-35f_min10x ucs table
path to this script: /share/compling/projects/sanpi/script/transform_ucs.sh
Sat May 25 21:35:10 EDT 2024
(TMP: /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/tmp/tmp_ANYmirror-20240525-213510.TrigBigr_frq-thrMIN-7-35f_min10x)
## Initial Contingency Info

DATA SET FILE:  /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/TrigBigr_frq-thrMIN-7.35f_min10x.ds.gz

# Frequency signatures computed by the ucs-make-tables tool for relational cooccurrences.
# Sample size:  N = 1761853 tokens,  V = 395338 pair types.
# A frequency

# %% [markdown]

### Define dictionary containing relevant vocab sizes
 !!! Warning This is a `#HACK`: \
     Rather than developing a command/code to retrieve the vocab sizes programmatically,
     I simply copied the values given in the log output of `transform_usc.sh`
     for each `PAT_DIR`+`UNIT` combination

 |      |   ANYmirror |   NEGmirror |   POSmirror |
 |:-----|------------:|------------:|------------:|
 | Bigr |     395,338 |      64,644 |     330,694 |
 | Adv  |      23,125 |       5,004 |      18,121 |
 | Adj  |      83,422 |      21,562 |      61,860 |

In [ ]:
VOCABS = {'ANYmirror': {'Adv': 23125, 'Bigr': 395338, 'Adj': 83422},
          'NEGmirror': {'Adv': 5004, 'Bigr': 64644, 'Adj': 21562},
          'POSmirror': {'Adv': 18121, 'Bigr': 330694, 'Adj': 61860}
          }  # HACK
VOCAB = VOCABS[PAT_DIR][UNIT]
pd.DataFrame(VOCABS)

,ANYmirror,NEGmirror,POSmirror
Adv,23125,5004,18121
Bigr,395338,64644,330694
Adj,83422,21562,61860


# %% [markdown]

## 4. Run `ucs_to_csv()` to convert `ucs/[PAT_DIR]/readable/*.txt` to format that `pandas` can parse as a dataframe

In [ ]:
! head -5 {readable}
csv_path = ucs2csv(readable)
print(f'CSV: `{csv_path.relative_to(RESULT_DIR)}`')

        l1  l2                                f           E11  am.log.likelihood  am.odds.ratio.disc    am.p1.given2     am.p2.given1  am.p1.given2.simple  am.p2.given1.simple      f1     f2        N  
----------  ----------------------------  -----  ------------  -----------------  ------------------  --------------  ---------------  -------------------  -------------------  ------  -----  -------  
    hardly  strictly_bluegrass               10     0.0310355        115.5224117        3.8297658232   0.99690210762    1.8288222e-03        1.00000000000        1.8288222e-03    5468     10  1761853  
 everybody  super_heady                      20     0.0904162        216.0126120        3.9566646052   0.99549049200    2.5109856e-03        1.00000000000        2.5109856e-03    7965     20  1761853  
        no  longer_viable                    56     0.2825661        592.7414834        4.3506693418   0.99498580143    6.2992126e-03        1.00000000000        6.2992126e-03    8890     56  

# %% [markdown]

##

In [ ]:
trig_amdf = pd.read_csv(csv_path).convert_dtypes()
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
0,hardly,strictly_bluegrass,10,0.03,115.52,3.83,1.00,0.00,1.00,0.00,5468,10,1761853
1,everybody,super_heady,20,0.09,216.01,3.96,1.00,0.00,1.00,0.00,7965,20,1761853
2,no,longer_viable,56,0.28,592.74,4.35,0.99,0.01,1.00,0.01,8890,56,1761853
3,no,longer_pink,42,0.21,444.49,4.23,0.99,0.00,1.00,0.00,8890,42,1761853
4,no,longer_afraid,18,0.09,190.45,3.86,0.99,0.00,1.00,0.00,8890,18,1761853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22087,or,very_excited,16,461.78,-876.17,-1.53,-0.18,-0.00,0.01,0.00,326614,2491,1761853
22088,or,too_aware,13,420.44,-809.02,-1.58,-0.18,-0.00,0.01,0.00,326614,2268,1761853
22089,or,too_common,19,915.97,"-1,836.19",-1.76,-0.18,-0.00,0.00,0.00,326614,4941,1761853
22090,or,very_special,18,898.73,"-1,806.84",-1.78,-0.18,-0.00,0.00,0.00,326614,4848,1761853


In [ ]:
# %%

trig_amdf['key'] = (trig_amdf.l1 + '~' +
                    trig_amdf.l2).astype('string')
trig_amdf = trig_amdf.set_index('key')
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
key,,,,,,,,,,,,,
hardly~strictly_bluegrass,hardly,strictly_bluegrass,10,0.03,115.52,3.83,1.00,0.00,1.00,0.00,5468,10,1761853
everybody~super_heady,everybody,super_heady,20,0.09,216.01,3.96,1.00,0.00,1.00,0.00,7965,20,1761853
no~longer_viable,no,longer_viable,56,0.28,592.74,4.35,0.99,0.01,1.00,0.01,8890,56,1761853
no~longer_pink,no,longer_pink,42,0.21,444.49,4.23,0.99,0.00,1.00,0.00,8890,42,1761853
no~longer_afraid,no,longer_afraid,18,0.09,190.45,3.86,0.99,0.00,1.00,0.00,8890,18,1761853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
or~very_excited,or,very_excited,16,461.78,-876.17,-1.53,-0.18,-0.00,0.01,0.00,326614,2491,1761853
or~too_aware,or,too_aware,13,420.44,-809.02,-1.58,-0.18,-0.00,0.01,0.00,326614,2268,1761853
or~too_common,or,too_common,19,915.97,"-1,836.19",-1.76,-0.18,-0.00,0.00,0.00,326614,4941,1761853


# %% [markdown]

## 6. Save to `./results/assoc_df/`

In [ ]:
df_csv_path = AM_DF_DIR.joinpath(
    str(csv_path.relative_to(UCS_DIR))
    .replace('/readable', '')
    .replace('.rsort-view_am-only', ''))

if not df_csv_path.is_file():
    confirm_dir(df_csv_path.parent)
    trig_amdf.to_csv(df_csv_path)

df_pkl_path = df_csv_path.with_suffix('.pkl.gz')
if not df_pkl_path.is_file():
    trig_amdf.to_pickle(df_csv_path.with_suffix('.pkl.gz'))

# %% [markdown]

## 7. Add additional AM via `add_extra_am()`

In [ ]:
ex_trig_amdf = add_extra_am(df=trig_amdf,
                            verbose=True,
                            vocab=VOCAB,
                            metrics=['t_score', 'mutual_information']
                            ).convert_dtypes()


Preview of Extended Measures (rounded)

| key                       |   t_score |   mutual_information |   deltaP_min |   deltaP_max |   deltaP_max_abs |   deltaP_product |   unexpected_f |   unexpected_ratio |
|:--------------------------|----------:|---------------------:|-------------:|-------------:|-----------------:|-----------------:|---------------:|-------------------:|
| hardly~strictly_bluegrass |      3.15 |                 2.51 |         0.00 |         1.00 |             1.00 |             0.00 |           9.97 |               1.00 |
| everybody~super_heady     |      4.45 |                 2.34 |         0.00 |         1.00 |             1.00 |             0.00 |          19.91 |               1.00 |
| no~longer_viable          |      7.45 |                 2.30 |         0.01 |         0.99 |             0.99 |             0.01 |          55.72 |               0.99 |
| no~longer_pink            |      6.45 |                 2.30 |         0.00 |         0.99 |          

#%% [markdown]

 Add trigger features as columns: polarity and quantification type

In [ ]:
def add_feature(triggers:pd.Series, 
                group_dict: dict) -> pd.Series:
    assign_dict = invert_set_dict(group_dict)
    return triggers.apply(lambda x: assign_dict[x] if x in assign_dict.keys() else '')

ex_trig_amdf['polarity'] = add_feature(ex_trig_amdf.l1, TRIGGER_POLARITY)
ex_trig_amdf['quant'] = add_feature(ex_trig_amdf.l1, TRIGGER_QUANT)

# %% [markdown]

 Save extended AM tables to `extra/` subdirectory if not already saved

In [ ]:
df_extra_csv = df_csv_path.parent / 'extra' / \
    df_csv_path.name.replace('.csv', '_extra.csv')
print(df_extra_csv)
if not df_extra_csv.is_file():
    confirm_dir(df_extra_csv.parent)
    ex_trig_amdf.to_csv(df_extra_csv)

df_extra_pkl = df_extra_csv.with_suffix('.pkl.gz')
if not df_extra_pkl.is_file():
    ex_trig_amdf.to_pickle(df_extra_pkl)

/share/compling/projects/sanpi/results/assoc_df/trigger_eval/ANYmirror/extra/TrigBigr_frq-thrMIN-7.35f_min10x_extra.csv


In [ ]:
# %%

ex_trig_full = ex_trig_amdf.copy()
ex_trig_abbr = adjust_assoc_columns(
    ex_trig_amdf[[c for c in ['polarity', 'quant'] + FOCUS if c in ex_trig_amdf.columns]]).sort_values('LRC', ascending=False)
cols = ex_trig_abbr.columns

# %% [markdown]

 Define lexical items with given lean shown in binary environment evaluation

In [ ]:
pos_prone = {
    'Adj': [
        'unrelated',
        'unable',
        'akin',
        'larger',
        'different',
        'familiar',
        'similar',
        'likely', 
        'brief', 
        'unaware'
    ],
    'Adv': [
        'slightly', 
        'definitely',
        'utterly',
        # LRC top
        'pretty', 
        'rather', 
        'plain', 
        'fairly', 
        'somewhat', 
        'otherwise', 
        'downright',
        'relatively', 
        # G2 top
        # 'very', 
        # 'even', 
        # 'just', 
        # dP1 top (and odds ratio disc)
        'plain', 
        'maybe'
    ], 
    'Bigr': [
        # G2 top
        'completely_different', 
        'too_familiar',
        'even_better',
        # dP1 top
        'quite_different', 
        'too_real', 
        'well_aware', 
        # LRC top
        'too_common', 
        'entirely_different'
        ]}
neg_prone = {
    'Bigr': [
        # LRC top
        'quite_sure',
        'really_sure', 
        'too_early', 
        'too_pleased', 
        'too_fancy',
        # dP1 top
        'entirely_sure', 
        'ever_easy', 
        'ever_perfect', 
        'particularly_surprising',
        'particularly_new',
        # G2 top
        'too_late', 
        'more_important',
        'so_easy', 
        'as_good', 
        'too_old'
        ], 
    'Adv': [
        'yet',
        #LRC top
        'ever',
        'any', 
        'longer', 
        'necessarily',
        'that',
        #dP1 top
        'before', 
        'wise',  #? How is this used as an adverb?
        'earthly',
        'remotely',
        'exactly',
        # G2 top
        'particularly', 
        'too',
        # 'inherently'
        ],
    'Adj': [
        # LRC top
        'early', 
        'late', 
        'fancy', 
        'alone', 
        'sure',
        # dP1 top
        'shabby',
        'demoralizing',
        'alone',
        'aggravating',
        'groundbreaking',
        'eventful',
        #G2 top
        'important', 
        'frustrating',
        'evident', 
        'certain'
        ]
    }
def sort_prone_by_f2(prone_list, amdf):
    return amdf.copy().loc[amdf.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
pos_prone[UNIT] = sort_prone_by_f2(pos_prone[UNIT], ex_trig_abbr)
neg_prone[UNIT] = sort_prone_by_f2(neg_prone[UNIT], ex_trig_abbr)
pos_prone[UNIT]

['completely_different',
 'too_familiar',
 'even_better',
 'too_common',
 'entirely_different',
 'quite_different',
 'too_real',
 'well_aware']

# %% [markdown]

## Strongest associations for each polarity by metric

In [ ]:
def show_metric_top(amdf: pd.DataFrame,
                    metric: str,
                    k=5, cols=[None]):
    if not any(cols):
        cols = amdf.columns
    return (pd.concat((polar_df.nlargest(k, metric)
                       for pol, polar_df in amdf.groupby('polarity')))
            .loc[:, [metric] + cols[cols != metric].to_list()]
            .reset_index(drop=True).set_index(['polarity', 'l1', 'l2'])
            )
def update_prone(exdf, pos_prone, neg_prone) -> dict: 
    prone_dict = {'positive':pos_prone, 
                  'negative': neg_prone}
    for polarity, prone in prone_dict.items():
        try: 
            polar_ex = exdf.loc[polarity, :]
        except KeyError: 
            continue
        else: 
                
            for topw in polar_ex.reset_index().l2.head(2).squeeze(): 
                if topw not in prone[UNIT]: 
                    prone[UNIT].append(topw)
            prone_dict[polarity] = prone
    return prone_dict['positive'], prone_dict['negative']

# %% [markdown]

 Top consevative log ratio $LRC$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr,"LRC")
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

LRC             quant     f  unexp_f  \
polarity l1         l2                                                       
negative none       too_pleased      8.80         not_exist  1272 1,248.12   
         hardly     more_different   8.66  hedged_not_exist   491   488.69   
         nobody     more_surprised   8.16         not_exist   166   165.14   
         barely     even_noticeable  8.08  hedged_not_exist    26    25.96   
         no         earthly_good     7.74         not_exist    73    72.59   
positive all        too_common       7.90         universal  4882 4,182.91   
                    too_familiar     7.65         universal  6933 5,934.39   
         everything all_right        7.45         universal  2166 2,108.54   
         all        too_aware        7.11         universal  2229 1,908.11   
         everything humanly_possible 6.62         universal   252   245.36   

                                      dP1  dP2  dP1_simple  dP2_simple  \
polarity l1         l2                                                   
negative none       too_pleased      0.92 0.04        0.94        0.04   
         hardly     more_different   0.66 0.09        0.66        0.09   
         nobody     more_surprised   0.69 0.03        0.69        0.03   
         barely     even_noticeable  0.46 0.02        0.46        0.02   
         no         earthly_good     0.89 0.01        0.89        0.01   
positive all        too_common       0.85 0.02        0.99        0.02   
                    too_familiar     0.84 0.03        0.98        0.03   
         everything all_right        0.82 0.05        0.85        0.05   
         all        too_aware        0.84 0.01        0.98        0.01   
         everything humanly_possible 0.83 0.01        0.85        0.01   

                                            G2        N      f1    f2  exp_f  
polarity l1         l2                                                        
negative none       too_pleased       9,701.23  1761853   31021  1356  23.88  
         hardly     more_different    4,764.11  1761853    5468   744   2.31  
         nobody     more_surprised    1,580.50  1761853    6311   239   0.86  
         barely     even_noticeable     300.92  1761853    1235    56   0.04  
         no         earthly_good        716.17  1761853    8890    82   0.41  
positive all        too_common       18,554.58  1761853  249279  4941 699.09  
                    too_familiar     26,063.93  1761853  249279  7058 998.61  
         everything all_right        14,379.56  1761853   39516  2562  57.46  
         all        too_aware         8,352.56  1761853  249279  2268 320.89  
         everything humanly_possible  1,668.60  1761853   39516   296   6.64

# %% [markdown]

 Top $\Delta P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1             quant   f  unexp_f  \
polarity l1         l2                                                       
negative hardly     strictly_bluegrass 1.00  hedged_not_exist  10     9.97   
         no         longer_viable      0.99         not_exist  56    55.72   
                    longer_pink        0.99         not_exist  42    41.79   
                    longer_afraid      0.99         not_exist  18    17.91   
                    longer_functional  0.99         not_exist  15    14.92   
positive everybody  super_heady        1.00         universal  20    19.91   
         everything generally_real     0.98         universal  18    17.60   
                    usually_accurate   0.98         universal  14    13.69   
                    generally_genuine  0.98         universal  11    10.75   
                    usually_correct    0.98         universal  10     9.78   

                                        LRC  dP2  dP1_simple  dP2_simple  \
polarity l1         l2                                                     
negative hardly     strictly_bluegrass 3.04 0.00        1.00        0.00   
         no         longer_viable      5.15 0.01        1.00        0.01   
                    longer_pink        4.71 0.00        1.00        0.00   
                    longer_afraid      3.36 0.00        1.00        0.00   
                    longer_functional  3.05 0.00        1.00        0.00   
positive everybody  super_heady        3.69 0.00        1.00        0.00   
         everything generally_real     1.18 0.00        1.00        0.00   
                    usually_accurate   0.76 0.00        1.00        0.00   
                    generally_genuine  0.33 0.00        1.00        0.00   
                    usually_correct    0.16 0.00        1.00        0.00   

                                           G2        N     f1  f2  exp_f  
polarity l1         l2                                                    
negative hardly     strictly_bluegrass 115.52  1761853   5468  10   0.03  
         no         longer_viable      592.74  1761853   8890  56   0.28  
                    longer_pink        444.49  1761853   8890  42   0.21  
                    longer_afraid      190.45  1761853   8890  18   0.09  
                    longer_functional  158.70  1761853   8890  15   0.08  
positive everybody  super_heady        216.01  1761853   7965  20   0.09  
         everything generally_real     136.71  1761853  39516  18   0.40  
                    usually_accurate   106.33  1761853  39516  14   0.31  
                    generally_genuine   83.55  1761853  39516  11   0.25  
                    usually_correct     75.95  1761853  39516  10   0.22

# %% [markdown]

 Top conditional probability $P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1_simple             quant    f  \
polarity l1        l2                                                           
negative no        longer_viable                  1.00         not_exist   56   
                   longer_pink                    1.00         not_exist   42   
                   longer_afraid                  1.00         not_exist   18   
         never     before_available               1.00         not_exist  175   
         no        longer_functional              1.00         not_exist   15   
positive everybody super_heady                    1.00         universal   20   
         many      instantly_bookable             1.00  hedged_universal   31   
         or        otherwise_inappropriate        1.00       existential  199   
                   otherwise_objectionable        1.00       existential  193   
         all       too_commonplace                1.00         universal  124   

                                            unexp_f  LRC  dP1  dP2  \
polarity l1        l2                                                
negative no        longer_viable              55.72 5.15 0.99 0.01   
                   longer_pink                41.79 4.71 0.99 0.00   
                   longer_afraid              17.91 3.36 0.99 0.00   
         never     before_available          164.10 3.13 0.94 0.00   
         no        longer_functional          14.92 3.05 0.99 0.00   
positive everybody super_heady                19.91 3.69 1.00 0.00   
         many      instantly_bookable         30.17 1.80 0.97 0.00   
         or        otherwise_inappropriate   162.11 1.54 0.81 0.00   
                   otherwise_objectionable   157.22 1.50 0.81 0.00   
         all       too_commonplace           106.46 1.31 0.86 0.00   

                                            dP2_simple     G2        N  \
polarity l1        l2                                                    
negative no        longer_viable                  0.01 592.74  1761853   
                   longer_pink                    0.00 444.49  1761853   
                   longer_afraid                  0.00 190.45  1761853   
         never     before_available               0.00 971.92  1761853   
         no        longer_functional              0.00 158.70  1761853   
positive everybody super_heady                    0.00 216.01  1761853   
         many      instantly_bookable             0.00 224.53  1761853   
         or        otherwise_inappropriate        0.00 670.87  1761853   
                   otherwise_objectionable        0.00 650.64  1761853   
         all       too_commonplace                0.00 485.03  1761853   

                                                f1   f2  exp_f  
polarity l1        l2                                           
negative no        longer_viable              8890   56   0.28  
                   longer_pink                8890   42   0.21  
                   longer_afraid              8890   18   0.09  
         never     before_available         109723  175  10.90  
         no        longer_functional          8890   15   0.08  
positive everybody super_heady                7965   20   0.09  
         many      instantly_bookable        47133   31   0.83  
         or        otherwise_inappropriate  326614  199  36.89  
                   otherwise_objectionable  326614  193  35.78  
         all       too_commonplace          249279  124  17.54

# %% [markdown]

 Top $\Delta P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2             quant      f  unexp_f  \
polarity l1         l2                                                      
negative nobody     quite_sure     0.10         not_exist    660   649.31   
         never      too_late       0.10         not_exist  10770 9,974.22   
         hardly     more_different 0.09  hedged_not_exist    491   488.69   
         scarcely   more_different 0.08  hedged_not_exist     67    66.67   
         nothing    more_important 0.06         not_exist   7208 5,913.61   
positive somethings more_likely    0.07       existential     12    11.27   
         everything all_right      0.05         universal   2166 2,108.54   
         something  as_simple      0.04       existential  12233 9,350.33   
         all        too_familiar   0.03         universal   6933 5,934.39   
         someone    so_young       0.03       existential    811   789.99   

                                    LRC  dP1  dP1_simple  dP2_simple  \
polarity l1         l2                                                 
negative nobody     quite_sure     5.96 0.22        0.22        0.10   
         never      too_late       6.14 0.79        0.84        0.10   
         hardly     more_different 8.66 0.66        0.66        0.09   
         scarcely   more_different 6.78 0.09        0.09        0.09   
         nothing    more_important 2.82 0.26        0.31        0.07   
positive somethings more_likely    1.77 0.00        0.00        0.08   
         everything all_right      7.45 0.82        0.85        0.05   
         something  as_simple      3.75 0.59        0.77        0.04   
         all        too_familiar   7.65 0.84        0.98        0.03   
         someone    so_young       6.28 0.63        0.65        0.03   

                                          G2        N      f1     f2    exp_f  
polarity l1         l2                                                         
negative nobody     quite_sure      4,366.80  1761853    6311   2984    10.69  
         never      too_late       49,945.41  1761853  109723  12778   795.78  
         hardly     more_different  4,764.11  1761853    5468    744     2.31  
         scarcely   more_different    590.05  1761853     785    744     0.33  
         nothing    more_important 15,088.39  1761853   98788  23085 1,294.39  
positive somethings more_likely        45.63  1761853     157   8144     0.73  
         everything all_right      14,379.56  1761853   39516   2562    57.46  
         something  as_simple      26,354.21  1761853  318343  15954 2,882.67  
         all        too_familiar   26,063.93  1761853  249279   7058   998.61  
         someone    so_young        5,044.83  1761853   29662   1248    21.01

# %% [markdown]

 Top conditional probability $P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2_simple             quant      f  \
polarity l1         l2                                                    
negative nobody     quite_sure            0.10         not_exist    660   
         never      too_late              0.10         not_exist  10770   
         hardly     more_different        0.09  hedged_not_exist    491   
         scarcely   more_different        0.09  hedged_not_exist     67   
         nothing    more_important        0.07         not_exist   7208   
positive somethings more_likely           0.08       existential     12   
         everything all_right             0.05         universal   2166   
         something  as_simple             0.04       existential  12233   
         all        too_familiar          0.03         universal   6933   
         someone    so_young              0.03       existential    811   

                                    unexp_f  LRC  dP1  dP2  dP1_simple  \
polarity l1         l2                                                   
negative nobody     quite_sure       649.31 5.96 0.22 0.10        0.22   
         never      too_late       9,974.22 6.14 0.79 0.10        0.84   
         hardly     more_different   488.69 8.66 0.66 0.09        0.66   
         scarcely   more_different    66.67 6.78 0.09 0.08        0.09   
         nothing    more_important 5,913.61 2.82 0.26 0.06        0.31   
positive somethings more_likely       11.27 1.77 0.00 0.07        0.00   
         everything all_right      2,108.54 7.45 0.82 0.05        0.85   
         something  as_simple      9,350.33 3.75 0.59 0.04        0.77   
         all        too_familiar   5,934.39 7.65 0.84 0.03        0.98   
         someone    so_young         789.99 6.28 0.63 0.03        0.65   

                                          G2        N      f1     f2    exp_f  
polarity l1         l2                                                         
negative nobody     quite_sure      4,366.80  1761853    6311   2984    10.69  
         never      too_late       49,945.41  1761853  109723  12778   795.78  
         hardly     more_different  4,764.11  1761853    5468    744     2.31  
         scarcely   more_different    590.05  1761853     785    744     0.33  
         nothing    more_important 15,088.39  1761853   98788  23085 1,294.39  
positive somethings more_likely        45.63  1761853     157   8144     0.73  
         everything all_right      14,379.56  1761853   39516   2562    57.46  
         something  as_simple      26,354.21  1761853  318343  15954 2,882.67  
         all        too_familiar   26,063.93  1761853  249279   7058   998.61  
         someone    so_young        5,044.83  1761853   29662   1248    21.01

# %% [markdown]

 Top log-likelihood $G^2$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'G2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

G2        quant      f  \
polarity l1        l2                                                   
negative never     too_late             49,945.41    not_exist  10770   
                   too_early            25,284.55    not_exist   4890   
         nothing   more_important       15,088.39    not_exist   7208   
         none      too_pleased           9,701.23    not_exist   1272   
         never     so_easy               9,479.17    not_exist   2501   
positive something as_simple            26,354.21  existential  12233   
         all       too_familiar         26,063.93    universal   6933   
                   too_easy             20,628.24    universal   6505   
         something completely_different 19,710.60  existential   7372   
         all       too_common           18,554.58    universal   4882   

                                         unexp_f  LRC  dP1  dP2  dP1_simple  \
polarity l1        l2                                                         
negative never     too_late             9,974.22 6.14 0.79 0.10        0.84   
                   too_early            4,568.77 7.59 0.89 0.04        0.95   
         nothing   more_important       5,913.61 2.82 0.26 0.06        0.31   
         none      too_pleased          1,248.12 8.80 0.92 0.04        0.94   
         never     so_easy              2,272.01 4.71 0.62 0.02        0.68   
positive something as_simple            9,350.33 3.75 0.59 0.04        0.77   
         all       too_familiar         5,934.39 7.65 0.84 0.03        0.98   
                   too_easy             5,466.20 5.26 0.75 0.03        0.89   
         something completely_different 5,862.91 4.81 0.71 0.02        0.88   
         all       too_common           4,182.91 7.90 0.85 0.02        0.99   

                                         dP2_simple        N      f1     f2  \
polarity l1        l2                                                         
negative never     too_late                    0.10  1761853  109723  12778   
                   too_early                   0.04  1761853  109723   5158   
         nothing   more_important              0.07  1761853   98788  23085   
         none      too_pleased                 0.04  1761853   31021   1356   
         never     so_easy                     0.02  1761853  109723   3677   
positive something as_simple                   0.04  1761853  318343  15954   
         all       too_familiar                0.03  1761853  249279   7058   
                   too_easy                    0.03  1761853  249279   7342   
         something completely_different        0.02  1761853  318343   8352   
         all       too_common                  0.02  1761853  249279   4941   

                                           exp_f  
polarity l1        l2                             
negative never     too_late               795.78  
                   too_early              321.23  
         nothing   more_important       1,294.39  
         none      too_pleased             23.88  
         never     so_easy                228.99  
positive something as_simple            2,882.67  
         all       too_familiar           998.61  
                   too_easy             1,038.80  
         something completely_different 1,509.09  
         all       too_common             699.09

In [ ]:
# %%

sig_trig_abbr = ex_trig_abbr.loc[ex_trig_abbr.LRC.abs() > 1, :]
def show_prone_ex(amdf, prone_list, list_index:int=0):
    try: 
        example = prone_list[list_index]

    except IndexError: 
        return list_index + 1, f'No {list_index}th entry'
    else:
        print(f'>> {example} <<')
        return list_index+ 1, amdf.filter(regex=f'~{example}$', axis=0).iloc[:10, :]

# %% [markdown]

 ---
 Significant Examples for Items demonstrated positive polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone = pos_prone[UNIT]
prone

['completely_different',
 'too_familiar',
 'even_better',
 'too_common',
 'entirely_different',
 'quite_different',
 'too_real',
 'well_aware',
 'super_heady',
 'generally_real',
 'instantly_bookable',
 'more_likely',
 'all_right',
 'as_simple']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> completely_different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~completely_different,positive,existential,7372,"5,862.91",4.81,0.71,0.02,0.88,0.02,"19,710.60",1761853,318343,8352,"1,509.09",something,completely_different
both~completely_different,positive,universal,42,-195.06,-1.26,-0.02,-0.00,0.01,0.00,-250.18,1761853,50007,8352,237.06,both,completely_different
everyone~completely_different,positive,universal,11,-142.24,-1.35,-0.02,-0.00,0.00,0.00,-229.63,1761853,32327,8352,153.24,everyone,completely_different
often~completely_different,positive,hedged_universal,57,-380.23,-1.92,-0.05,-0.00,0.01,0.00,-547.83,1761853,92233,8352,437.23,often,completely_different
many~completely_different,positive,hedged_universal,10,-213.43,-1.92,-0.03,-0.00,0.00,0.00,-371.29,1761853,47133,8352,223.43,many,completely_different
sometimes~completely_different,positive,existential,38,-316.15,-1.94,-0.04,-0.00,0.00,0.00,-476.39,1761853,74707,8352,354.15,sometimes,completely_different
all~completely_different,positive,universal,207,-974.70,-2.12,-0.12,-0.00,0.02,0.00,"-1,359.54",1761853,249279,8352,"1,181.70",all,completely_different
or~completely_different,positive,existential,214,"-1,334.30",-2.55,-0.16,-0.01,0.03,0.00,"-2,074.41",1761853,326614,8352,"1,548.30",or,completely_different
always~completely_different,positive,universal,14,-375.67,-2.67,-0.05,-0.00,0.00,0.00,-677.46,1761853,82200,8352,389.67,always,completely_different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_familiar <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~too_familiar,positive,universal,6933,"5,934.39",7.65,0.84,0.03,0.98,0.03,"26,063.93",1761853,249279,7058,998.61,all,too_familiar
many~too_familiar,positive,hedged_universal,15,-173.82,-1.57,-0.02,-0.00,0.00,0.00,-276.67,1761853,47133,7058,188.82,many,too_familiar
never~too_familiar,negative,not_exist,15,-424.55,-2.84,-0.06,-0.00,0.00,0.00,-776.20,1761853,109723,7058,439.55,never,too_familiar
some~too_familiar,positive,existential,11,-449.41,-3.01,-0.06,-0.00,0.00,0.00,-848.49,1761853,114929,7058,460.41,some,too_familiar
something~too_familiar,positive,existential,18,"-1,257.29",-4.49,-0.18,-0.00,0.00,0.00,"-2,622.73",1761853,318343,7058,"1,275.29",something,too_familiar
or~too_familiar,positive,existential,16,"-1,292.42",-4.59,-0.18,-0.00,0.00,0.00,"-2,721.11",1761853,326614,7058,"1,308.42",or,too_familiar


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> even_better <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~even_better,positive,existential,4352,"3,220.62",3.22,0.53,0.01,0.71,0.01,"8,110.68",1761853,326614,6103,"1,131.38",or,even_better
both~even_better,positive,universal,25,-148.22,-1.18,-0.02,-0.00,0.00,0.00,-203.79,1761853,50007,6103,173.22,both,even_better
some~even_better,positive,existential,94,-304.11,-1.30,-0.05,-0.00,0.02,0.00,-353.65,1761853,114929,6103,398.11,some,even_better
many~even_better,positive,hedged_universal,14,-149.27,-1.38,-0.02,-0.00,0.00,0.00,-233.96,1761853,47133,6103,163.27,many,even_better
often~even_better,positive,hedged_universal,54,-265.49,-1.51,-0.04,-0.00,0.01,0.00,-351.80,1761853,92233,6103,319.49,often,even_better
always~even_better,positive,universal,10,-274.74,-2.30,-0.05,-0.00,0.00,0.00,-496.24,1761853,82200,6103,284.74,always,even_better
all~even_better,positive,universal,59,-804.49,-3.00,-0.13,-0.00,0.01,0.00,"-1,413.02",1761853,249279,6103,863.49,all,even_better


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_common <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~too_common,positive,universal,4882,"4,182.91",7.90,0.85,0.02,0.99,0.02,"18,554.58",1761853,249279,4941,699.09,all,too_common
or~too_common,positive,existential,19,-896.97,-3.99,-0.18,-0.00,0.00,0.00,"-1,836.19",1761853,326614,4941,915.97,or,too_common
something~too_common,positive,existential,11,-881.77,-4.15,-0.18,-0.00,0.00,0.00,"-1,849.27",1761853,318343,4941,892.77,something,too_common


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> entirely_different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~entirely_different,positive,existential,3115,"2,517.29",5.58,0.76,0.01,0.94,0.01,"9,289.50",1761853,318343,3308,597.71,something,entirely_different
often~entirely_different,positive,hedged_universal,15,-158.17,-1.48,-0.05,-0.00,0.00,0.00,-251.10,1761853,92233,3308,173.17,often,entirely_different
or~entirely_different,positive,existential,45,-568.24,-2.81,-0.17,-0.00,0.01,0.00,"-1,014.80",1761853,326614,3308,613.24,or,entirely_different
all~entirely_different,positive,universal,14,-454.04,-3.08,-0.14,-0.00,0.00,0.00,-879.78,1761853,249279,3308,468.04,all,entirely_different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> quite_different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~quite_different,positive,existential,2268,"1,697.03",3.20,0.54,0.01,0.72,0.01,"4,366.68",1761853,318343,3160,570.97,something,quite_different
always~quite_different,positive,universal,18,-129.43,-1.15,-0.04,-0.00,0.01,0.00,-188.85,1761853,82200,3160,147.43,always,quite_different
or~quite_different,positive,existential,34,-551.80,-2.97,-0.17,-0.00,0.01,0.00,"-1,021.82",1761853,326614,3160,585.80,or,quite_different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_real <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~too_real,positive,universal,2250,"1,920.19",6.47,0.82,0.01,0.97,0.01,"8,138.56",1761853,249279,2331,329.81,all,too_real
or~too_real,positive,existential,22,-410.12,-2.82,-0.18,-0.00,0.01,0.00,-772.67,1761853,326614,2331,432.12,or,too_real


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> well_aware <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everyone~well_aware,positive,universal,391,360.27,3.55,0.22,0.01,0.23,0.01,"1,357.91",1761853,32327,1675,30.73,everyone,well_aware
all~well_aware,positive,universal,821,584.01,2.14,0.35,0.00,0.49,0.00,"1,151.78",1761853,249279,1675,236.99,all,well_aware
everybody~well_aware,positive,universal,65,57.43,2.12,0.03,0.01,0.04,0.01,167.05,1761853,7965,1675,7.57,everybody,well_aware
many~well_aware,positive,hedged_universal,145,100.19,1.07,0.06,0.00,0.09,0.00,146.68,1761853,47133,1675,44.81,many,well_aware
or~well_aware,positive,existential,34,-276.51,-2.03,-0.17,-0.00,0.02,0.00,-455.50,1761853,326614,1675,310.51,or,well_aware
something~well_aware,positive,existential,24,-278.65,-2.24,-0.17,-0.00,0.01,0.00,-489.04,1761853,318343,1675,302.65,something,well_aware


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> super_heady <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everybody~super_heady,positive,universal,20,19.91,3.69,1.00,0.00,1.00,0.00,216.01,1761853,7965,20,0.09,everybody,super_heady


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> generally_real <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everything~generally_real,positive,universal,18,17.60,1.18,0.98,0.00,1.00,0.00,136.71,1761853,39516,18,0.40,everything,generally_real


# %% [markdown]

 ---
## Examples for Items demonstrating *Negative* polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone_list = neg_prone[UNIT]
prone = sig_trig_abbr.copy().loc[sig_trig_abbr.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
prone

['more_important',
 'too_late',
 'as_good',
 'too_early',
 'so_easy',
 'too_old',
 'quite_sure',
 'too_pleased',
 'really_sure',
 'more_different',
 'too_fancy',
 'particularly_new',
 'ever_easy',
 'entirely_sure',
 'ever_perfect',
 'particularly_surprising',
 'longer_viable',
 'longer_pink',
 'strictly_bluegrass']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> more_important <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~more_important,negative,not_exist,7208,"5,913.61",2.82,0.26,0.06,0.31,0.07,"15,088.39",1761853,98788,23085,"1,294.39",nothing,more_important
never~more_important,negative,not_exist,3853,"2,415.33",1.45,0.11,0.02,0.17,0.04,"3,104.12",1761853,109723,23085,"1,437.67",never,more_important
nor~more_important,negative,not_exist,37,-166.69,-1.12,-0.01,-0.01,0.00,0.00,-210.20,1761853,15546,23085,203.69,nor,more_important
some~more_important,positive,existential,321,"-1,184.88",-1.85,-0.05,-0.01,0.01,0.00,"-1,454.52",1761853,114929,23085,"1,505.88",some,more_important
or~more_important,positive,existential,1067,"-3,212.52",-1.97,-0.14,-0.01,0.05,0.00,"-4,020.10",1761853,326614,23085,"4,279.52",or,more_important
everything~more_important,positive,universal,61,-456.77,-2.06,-0.02,-0.01,0.00,0.00,-667.25,1761853,39516,23085,517.77,everything,more_important
both~more_important,positive,universal,44,-611.23,-2.70,-0.03,-0.01,0.00,0.00,"-1,009.05",1761853,50007,23085,655.23,both,more_important
everyone~more_important,positive,universal,15,-408.57,-2.73,-0.02,-0.01,0.00,0.00,-729.55,1761853,32327,23085,423.57,everyone,more_important
many~more_important,positive,hedged_universal,22,-595.57,-3.10,-0.03,-0.01,0.00,0.00,"-1,067.86",1761853,47133,23085,617.57,many,more_important


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_late <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~too_late,negative,not_exist,10770,"9,974.22",6.14,0.79,0.10,0.84,0.10,"49,945.41",1761853,109723,12778,795.78,never,too_late
always~too_late,positive,universal,150,-446.16,-1.37,-0.04,-0.01,0.01,0.00,-497.06,1761853,82200,12778,596.16,always,too_late
everything~too_late,positive,universal,30,-256.59,-1.79,-0.02,-0.01,0.00,0.00,-384.72,1761853,39516,12778,286.59,everything,too_late
or~too_late,positive,existential,588,"-1,780.80",-1.89,-0.14,-0.01,0.05,0.00,"-2,223.57",1761853,326614,12778,"2,368.80",or,too_late
many~too_late,positive,hedged_universal,12,-329.84,-2.50,-0.03,-0.01,0.00,0.00,-590.34,1761853,47133,12778,341.84,many,too_late
all~too_late,positive,universal,240,"-1,567.92",-2.57,-0.12,-0.01,0.02,0.00,"-2,392.25",1761853,249279,12778,"1,807.92",all,too_late
both~too_late,positive,universal,10,-352.68,-2.62,-0.03,-0.01,0.00,0.00,-646.04,1761853,50007,12778,362.68,both,too_late
nothing~too_late,negative,not_exist,18,-698.47,-3.46,-0.06,-0.01,0.00,0.00,"-1,309.26",1761853,98788,12778,716.47,nothing,too_late
some~too_late,positive,existential,16,-817.53,-3.74,-0.06,-0.01,0.00,0.00,"-1,569.54",1761853,114929,12778,833.53,some,too_late


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> as_good <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~as_good,negative,not_exist,1981,"1,615.37",2.71,0.28,0.02,0.34,0.02,"4,018.20",1761853,109723,5871,365.63,never,as_good
rarely~as_good,negative,hedged_not_exist,118,103.21,2.27,0.02,0.02,0.02,0.03,287.92,1761853,4439,5871,14.79,rarely,as_good
none~as_good,negative,not_exist,444,340.63,1.79,0.06,0.01,0.08,0.01,637.36,1761853,31021,5871,103.37,none,as_good
neither~as_good,negative,not_exist,92,70.48,1.26,0.01,0.01,0.02,0.01,127.99,1761853,6458,5871,21.52,neither,as_good
all~as_good,positive,universal,274,-556.67,-1.24,-0.10,-0.00,0.05,0.00,-566.34,1761853,249279,5871,830.67,all,as_good
something~as_good,positive,existential,377,-683.81,-1.25,-0.12,-0.00,0.06,0.00,-682.28,1761853,318343,5871,"1,060.81",something,as_good


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_early <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~too_early,negative,not_exist,4890,"4,568.77",7.59,0.89,0.04,0.95,0.04,"25,284.55",1761853,109723,5158,321.23,never,too_early
always~too_early,positive,universal,50,-190.65,-1.16,-0.04,-0.00,0.01,0.00,-231.93,1761853,82200,5158,240.65,always,too_early
sometimes~too_early,positive,existential,24,-194.71,-1.56,-0.04,-0.00,0.00,0.00,-291.47,1761853,74707,5158,218.71,sometimes,too_early
often~too_early,positive,hedged_universal,10,-260.02,-2.23,-0.05,-0.00,0.00,0.00,-468.49,1761853,92233,5158,270.02,often,too_early
or~too_early,positive,existential,99,-857.19,-2.71,-0.17,-0.00,0.02,0.00,"-1,432.19",1761853,326614,5158,956.19,or,too_early
all~too_early,positive,universal,30,-699.79,-3.31,-0.14,-0.00,0.01,0.00,"-1,315.56",1761853,249279,5158,729.79,all,too_early


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> so_easy <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~so_easy,negative,not_exist,2501,"2,272.01",4.71,0.62,0.02,0.68,0.02,"9,479.17",1761853,109723,3677,228.99,never,so_easy
often~so_easy,positive,hedged_universal,20,-172.49,-1.50,-0.05,-0.00,0.01,0.00,-263.15,1761853,92233,3677,192.49,often,so_easy
some~so_easy,positive,existential,32,-207.86,-1.53,-0.06,-0.00,0.01,0.00,-299.53,1761853,114929,3677,239.86,some,so_easy
something~so_easy,positive,existential,137,-527.38,-1.80,-0.14,-0.00,0.04,0.00,-710.59,1761853,318343,3677,664.38,something,so_easy
or~so_easy,positive,existential,42,-639.65,-3.03,-0.17,-0.00,0.01,0.00,"-1,174.53",1761853,326614,3677,681.65,or,so_easy


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_old <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~too_old,negative,not_exist,2218,"2,013.61",4.67,0.61,0.02,0.68,0.02,"8,356.35",1761853,109723,3282,204.39,never,too_old
either~too_old,positive,existential,40,32.76,1.18,0.01,0.01,0.01,0.01,71.84,1761853,3886,3282,7.24,either,too_old
always~too_old,positive,universal,16,-137.12,-1.26,-0.04,-0.00,0.00,0.00,-208.13,1761853,82200,3282,153.12,always,too_old
often~too_old,positive,hedged_universal,21,-150.81,-1.30,-0.05,-0.00,0.01,0.00,-220.80,1761853,92233,3282,171.81,often,too_old
all~too_old,positive,universal,96,-368.36,-1.60,-0.11,-0.00,0.03,0.00,-480.89,1761853,249279,3282,464.36,all,too_old


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> quite_sure <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nobody~quite_sure,negative,not_exist,660,649.31,5.96,0.22,0.10,0.22,0.10,"4,366.80",1761853,6311,2984,10.69,nobody,quite_sure
never~quite_sure,negative,not_exist,2176,"1,990.17",5.00,0.67,0.02,0.73,0.02,"8,739.05",1761853,109723,2984,185.83,never,quite_sure


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_pleased <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
none~too_pleased,negative,not_exist,1272,"1,248.12",8.80,0.92,0.04,0.94,0.04,"9,701.23",1761853,31021,1356,23.88,none,too_pleased


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> really_sure <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nobody~really_sure,negative,not_exist,285,281.73,6.40,0.31,0.04,0.31,0.05,"2,093.12",1761853,6311,914,3.27,nobody,really_sure
never~really_sure,negative,not_exist,559,502.08,4.01,0.55,0.00,0.61,0.01,"1,930.70",1761853,109723,914,56.92,never,really_sure


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> more_different <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
hardly~more_different,negative,hedged_not_exist,491,488.69,8.66,0.66,0.09,0.66,0.09,"4,764.11",1761853,5468,744,2.31,hardly,more_different
scarcely~more_different,negative,hedged_not_exist,67,66.67,6.78,0.09,0.08,0.09,0.09,590.05,1761853,785,744,0.33,scarcely,more_different


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too_fancy <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~too_fancy,negative,not_exist,521,490.55,6.85,0.90,0.01,0.96,0.01,"2,823.11",1761853,98788,543,30.45,nothing,too_fancy


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, [r'exactly\w*'] if UNIT!='Adj' else [r'un\w+'])
exdf

>> exactly\w* <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nobody~exactly_sure,negative,not_exist,65,64.47,6.41,0.44,0.01,0.44,0.01,530.42,1761853,6311,148,0.53,nobody,exactly_sure
none~exactly_new,negative,not_exist,18,17.47,3.33,0.58,0.00,0.60,0.00,105.48,1761853,31021,30,0.53,none,exactly_new
never~exactly_clear,negative,not_exist,45,41.76,3.26,0.80,0.00,0.87,0.00,209.68,1761853,109723,52,3.24,never,exactly_clear
neither~exactly_right,negative,not_exist,14,13.51,2.66,0.10,0.00,0.10,0.00,68.01,1761853,6458,135,0.49,neither,exactly_right
none~exactly_alike,negative,not_exist,23,21.49,2.35,0.25,0.00,0.27,0.00,88.19,1761853,31021,86,1.51,none,exactly_alike
everything~exactly_right,positive,universal,35,31.97,2.32,0.24,0.00,0.26,0.00,115.87,1761853,39516,135,3.03,everything,exactly_right
never~exactly_sure,negative,not_exist,67,57.78,2.28,0.39,0.00,0.45,0.00,178.61,1761853,109723,148,9.22,never,exactly_sure
none~exactly_right,negative,not_exist,21,18.62,1.41,0.14,0.00,0.16,0.00,57.02,1761853,31021,135,2.38,none,exactly_right
never~exactly_easy,negative,not_exist,16,14.75,1.32,0.74,0.00,0.80,0.00,69.34,1761853,109723,20,1.25,never,exactly_easy


In [ ]:
ix, exdf = show_prone_ex(sig_trig_abbr, [r'that\w*'])
exdf

>> that\w* <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
rarely~that_straightforward,negative,hedged_not_exist,14,13.88,4.74,0.29,0.00,0.29,0.00,109.81,1761853,4439,48,0.12,rarely,that_straightforward
rarely~that_simple,negative,hedged_not_exist,63,61.67,4.65,0.12,0.01,0.12,0.01,370.87,1761853,4439,529,1.33,rarely,that_simple
nobody~that_stupid,negative,not_exist,18,17.78,4.54,0.29,0.00,0.29,0.00,128.41,1761853,6311,62,0.22,nobody,that_stupid
never~that_easy,negative,not_exist,332,300.86,4.12,0.60,0.00,0.66,0.00,"1,227.51",1761853,109723,500,31.14,never,that_easy
never~that_great,negative,not_exist,214,192.83,3.76,0.57,0.00,0.63,0.00,756.46,1761853,109723,340,21.17,never,that_great
never~that_popular,negative,not_exist,60,55.64,3.69,0.79,0.00,0.86,0.00,277.04,1761853,109723,70,4.36,never,that_popular
never~that_simple,negative,not_exist,303,270.06,3.61,0.51,0.00,0.57,0.00,990.03,1761853,109723,529,32.94,never,that_simple
never~that_close,negative,not_exist,55,50.83,3.49,0.76,0.00,0.82,0.00,243.96,1761853,109723,67,4.17,never,that_close
never~that_good,negative,not_exist,327,288.70,3.43,0.47,0.00,0.53,0.00,"1,003.36",1761853,109723,615,38.30,never,that_good


In [ ]:
ix, exdf = show_prone_ex(sig_trig_abbr, [r'immediately\w*'])
exdf

>> immediately\w* <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nobody~immediately_available,negative,not_exist,58,57.02,5.01,0.21,0.01,0.21,0.01,372.50,1761853,6311,274,0.98,nobody,immediately_available
neither~immediately_available,negative,not_exist,27,26.00,3.23,0.09,0.00,0.10,0.00,128.41,1761853,6458,274,1.00,neither,immediately_available
none~immediately_available,negative,not_exist,51,46.18,2.40,0.17,0.00,0.19,0.00,156.67,1761853,31021,274,4.82,none,immediately_available
nor~immediately_clear,negative,not_exist,10,9.62,2.13,0.22,0.00,0.23,0.00,48.56,1761853,15546,43,0.38,nor,immediately_clear


In [ ]:
ix, exdf = show_prone_ex(sig_trig_abbr, [r'increasingly\w*'])
exdf

>> increasingly\w* <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
many~increasingly_concerned,positive,hedged_universal,18,17.36,2.90,0.72,0.00,0.75,0.00,103.70,1761853,47133,24,0.64,many,increasingly_concerned
many~increasingly_worried,positive,hedged_universal,10,9.55,1.65,0.56,0.00,0.59,0.00,49.77,1761853,47133,17,0.45,many,increasingly_worried


In [ ]:
ix, exdf = show_prone_ex(sig_trig_abbr, [r'definitely\w*'])
exdf

>> definitely\w* <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~definitely_wrong,positive,existential,359,293.23,4.65,0.81,0.00,0.99,0.00,"1,178.01",1761853,318343,364,65.77,something,definitely_wrong
some~definitely_better,positive,existential,43,38.63,2.59,0.58,0.00,0.64,0.00,150.60,1761853,114929,67,4.37,some,definitely_better
something~definitely_different,positive,existential,34,26.59,1.05,0.65,0.00,0.83,0.00,81.66,1761853,318343,41,7.41,something,definitely_different


In [ ]:
ix, exdf = show_prone_ex(sig_trig_abbr, [r'before\w*'])
exdf

>> before\w* <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~before_available,negative,not_exist,175,164.10,3.13,0.94,0.00,1.00,0.00,971.92,1761853,109723,175,10.90,never,before_available


In [ ]:
show_metric_top(sig_trig_abbr.filter(like='~very', axis=0), 'LRC', 10)

LRC        quant     f  unexp_f   dP1  \
polarity l1         l2                                                        
negative nothing    very_remarkable  2.36    not_exist    48    42.51  0.43   
         neither    very_appealing   1.95    not_exist    17    15.87  0.05   
                    very_attractive  1.67    not_exist    17    15.64  0.04   
                    very_good        1.56    not_exist   130   102.75  0.01   
         none       very_convincing  1.46    not_exist    19    17.05  0.15   
                    very_likeable    1.33    not_exist    13    11.91  0.19   
                    very_surprising  1.26    not_exist    25    21.53  0.11   
         neither    very_likely      1.18    not_exist    17    15.09  0.03   
         never      very_good        1.09    not_exist  1079   616.03  0.08   
                    very_nice       -1.17    not_exist    17  -127.05 -0.05   
positive something  very_wrong       5.52  existential  2555 2,064.80  0.76   
         always     very_responsive  4.94    universal   182   171.27  0.74   
         everyone   very_welcoming   4.50    universal   163   156.08  0.41   
         always     very_careful     4.46    universal   381   353.05  0.59   
         everything very_clean       4.44    universal   233   221.40  0.43   
         everyone   very_friendly    4.39    universal   616   582.88  0.32   
         someone    very_dear        4.27  existential   109   104.40  0.38   
         all        very_innovative  4.19    universal   368   308.43  0.73   
         something  very_special     4.10  existential  3998 3,122.03  0.65   
         always     very_prompt      3.84    universal    49    46.39  0.83   

                                      dP2  dP1_simple  dP2_simple       G2  \
polarity l1         l2                                                       
negative nothing    very_remarkable  0.00        0.49        0.00   146.56   
         neither    very_appealing   0.00        0.06        0.00    61.44   
                    very_attractive  0.00        0.05        0.00    55.39   
                    very_good        0.02        0.02        0.02   203.84   
         none       very_convincing  0.00        0.17        0.00    55.16   
                    very_likeable    0.00        0.21        0.00    43.09   
                    very_surprising  0.00        0.13        0.00    58.20   
         neither    very_likely      0.00        0.03        0.00    44.69   
         never      very_good        0.01        0.15        0.01   653.72   
                    very_nice       -0.00        0.01        0.00  -188.90   
positive something  very_wrong       0.01        0.94        0.01 7,617.47   
         always     very_responsive  0.00        0.79        0.00   884.99   
         everyone   very_welcoming   0.00        0.43        0.01   796.40   
         always     very_careful     0.00        0.64        0.00 1,572.46   
         everything very_clean       0.01        0.45        0.01 1,072.08   
         everyone   very_friendly    0.02        0.34        0.02 2,663.45   
         someone    very_dear        0.00        0.40        0.00   529.01   
         all        very_innovative  0.00        0.87        0.00 1,137.20   
         something  very_special     0.01        0.82        0.01 9,556.20   
         always     very_prompt      0.00        0.88        0.00   258.86   

                                           N      f1    f2  exp_f  
polarity l1         l2                                             
negative nothing    very_remarkable  1761853   98788    98   5.49  
         neither    very_appealing   1761853    6458   307   1.13  
                    very_attractive  1761853    6458   370   1.36  
                    very_good        1761853    6458  7434  27.25  
         none       very_convincing  1761853   31021   111   1.95  
                    very_likeable    1761853   31021    62   1.09  
                    very_surprising  1

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='~exactly', axis=0), 'LRC', 10)

LRC      quant   f  unexp_f  dP1  dP2  \
polarity l1         l2                                                     
negative nobody     exactly_sure  6.41  not_exist  65    64.47 0.44 0.01   
         none       exactly_new   3.33  not_exist  18    17.47 0.58 0.00   
         never      exactly_clear 3.26  not_exist  45    41.76 0.80 0.00   
         neither    exactly_right 2.66  not_exist  14    13.51 0.10 0.00   
         none       exactly_alike 2.35  not_exist  23    21.49 0.25 0.00   
         never      exactly_sure  2.28  not_exist  67    57.78 0.39 0.00   
         none       exactly_right 1.41  not_exist  21    18.62 0.14 0.00   
         never      exactly_easy  1.32  not_exist  16    14.75 0.74 0.00   
positive everything exactly_right 2.32  universal  35    31.97 0.24 0.00   

                                   dP1_simple  dP2_simple     G2        N  \
polarity l1         l2                                                      
negative nobody     exactly_sure         0.44        0.01 530.42  1761853   
         none       exactly_new          0.60        0.00 105.48  1761853   
         never      exactly_clear        0.87        0.00 209.68  1761853   
         neither    exactly_right        0.10        0.00  68.01  1761853   
         none       exactly_alike        0.27        0.00  88.19  1761853   
         never      exactly_sure         0.45        0.00 178.61  1761853   
         none       exactly_right        0.16        0.00  57.02  1761853   
         never      exactly_easy         0.80        0.00  69.34  1761853   
positive everything exactly_right        0.26        0.00 115.87  1761853   

                                       f1   f2  exp_f  
polarity l1         l2                                 
negative nobody     exactly_sure     6311  148   0.53  
         none       exactly_new     31021   30   0.53  
         never      exactly_clear  109723   52   3.24  
         neither    exactly_right    6458  135   0.49  
         none       exactly_alike   31021   86   1.51  
         never      exactly_sure   109723  148   9.22  
         none       exactly_right   31021  135   2.38  
         never      exactly_easy   109723   20   1.25  
positive everything exactly_right   39516  135   3.03

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='body~', axis=0), 'LRC', 10)

LRC      quant    f  unexp_f  dP1  dP2  \
polarity l1        l2                                                          
negative nobody    more_surprised    8.16  not_exist  166   165.14 0.69 0.03   
                   more_disappointed 6.52  not_exist   67    66.48 0.46 0.01   
                   more_frustrated   6.48  not_exist   56    55.58 0.48 0.01   
                   exactly_sure      6.41  not_exist   65    64.47 0.44 0.01   
                   really_sure       6.40  not_exist  285   281.73 0.31 0.04   
                   even_sure         6.01  not_exist   32    31.76 0.48 0.01   
                   ever_happy        5.98  not_exist   25    24.84 0.55 0.00   
                   quite_sure        5.96  not_exist  660   649.31 0.22 0.10   
                   seriously_hurt    5.90  not_exist   49    48.53 0.37 0.01   
                   more_committed    5.87  not_exist   28    27.79 0.48 0.00   
positive everybody all_excited       4.84  universal   33    32.50 0.29 0.00   
                   finally_equal     4.43  universal   11    10.91 0.57 0.00   
                   real_excited      4.41  universal   20    19.71 0.30 0.00   
                   really_excited    4.04  universal  151   144.71 0.10 0.02   
                   real_happy        3.93  universal   14    13.79 0.29 0.00   
                   pretty_excited    3.87  universal   55    53.11 0.13 0.01   
                   super_heady       3.69  universal   20    19.91 1.00 0.00   
                   so_excited        3.46  universal  161   151.12 0.07 0.02   
                   so_nice           3.35  universal  121   113.54 0.07 0.01   
                   pretty_happy      3.34  universal   38    36.42 0.10 0.00   

                                      dP1_simple  dP2_simple       G2  \
polarity l1        l2                                                   
negative nobody    more_surprised           0.69        0.03 1,580.50   
                   more_disappointed        0.46        0.01   554.53   
                   more_frustrated          0.48        0.01   469.71   
                   exactly_sure             0.44        0.01   530.42   
                   really_sure              0.31        0.05 2,093.12   
                   even_sure                0.48        0.01   269.41   
                   ever_happy               0.56        0.00   220.01   
                   quite_sure               0.22        0.10 4,366.80   
                   seriously_hurt           0.37        0.01   379.68   
                   more_committed           0.48        0.00   235.38   
positive everybody all_excited              0.30        0.00   222.08   
                   finally_equal            0.58        0.00    93.00   
                   real_excited             0.31        0.00   136.18   
                   really_excited           0.11        0.02   688.62   
                   real_happy               0.30        0.00    94.25   
                   pretty_excited           0.13        0.01   271.75   
                   super_heady              1.00        0.00   216.01   
                   so_excited               0.07        0.02   610.14   
                   so_nice                  0.07        0.02   456.88   
                   pretty_happy             0.11        0.00   173.08   

                                            N    f1    f2  exp_f  
polarity l1        l2                                             
negative nobody    more_surprised     1761853  6311   239   0.86  
                   more_disappointed  1761853  6311   146   0.52  
                   more_frustrated    1761853  6311   117   0.42  
                   exactly_sure       1761853  6311   148   0.53  
                   really_sure        1761853  6311   914   3.27  
                   even_sure          1761853  6311    66   0.24  
                   ever_happy         1761853  6311    45   0.16  
                   quite_sure         1761853  6311  2984  10.69  


In [ ]:
show_metric_top(sig_trig_abbr.filter(like='terribly', axis=0), 'LRC', 10)

LRC        quant     f  unexp_f  \
polarity l1        l2                                                       
negative nothing   terribly_new          4.00    not_exist    64    60.02   
         none      terribly_surprising   3.67    not_exist    32    30.82   
         nothing   terribly_exciting     3.30    not_exist    71    64.83   
                   terribly_original     3.13    not_exist    38    35.42   
         none      terribly_interesting  2.26    not_exist    19    17.84   
         nothing   terribly_unusual      1.90    not_exist    20    18.65   
         never     terribly_interested   1.73    not_exist    24    21.32   
                   terribly_good         1.62    not_exist    21    18.76   
         nothing   terribly_interesting  1.59    not_exist    28    24.30   
                   terribly_surprising   1.46    not_exist    27    23.24   
positive something terribly_wrong        4.98  existential  1571 1,262.57   
         all       terribly_wrong       -1.95    universal    20  -221.52   
         or        terribly_wrong       -2.50  existential    17  -299.45   

                                          dP1   dP2  dP1_simple  dP2_simple  \
polarity l1        l2                                                         
negative nothing   terribly_new          0.85  0.00        0.90        0.00   
         none      terribly_surprising   0.46  0.00        0.48        0.00   
         nothing   terribly_exciting     0.59  0.00        0.65        0.00   
                   terribly_original     0.77  0.00        0.83        0.00   
         none      terribly_interesting  0.27  0.00        0.29        0.00   
         nothing   terribly_unusual      0.78  0.00        0.83        0.00   
         never     terribly_interested   0.50  0.00        0.56        0.00   
                   terribly_good         0.52  0.00        0.58        0.00   
         nothing   terribly_interesting  0.37  0.00        0.42        0.00   
                   terribly_surprising   0.35  0.00        0.40        0.00   
positive something terribly_wrong        0.74  0.00        0.92        0.00   
         all       terribly_wrong       -0.13 -0.00        0.01        0.00   
         or        terribly_wrong       -0.18 -0.00        0.01        0.00   

                                              G2        N      f1    f2  exp_f  
polarity l1        l2                                                           
negative nothing   terribly_new           323.91  1761853   98788    71   3.98  
         none      terribly_surprising    167.05  1761853   31021    67   1.18  
         nothing   terribly_exciting      270.62  1761853   98788   110   6.17  
                   terribly_original      177.40  1761853   98788    46   2.58  
         none      terribly_interesting    75.95  1761853   31021    66   1.16  
         nothing   terribly_unusual        94.08  1761853   98788    24   1.35  
         never     terribly_interested     76.68  1761853  109723    43   2.68  
                   terribly_good           69.63  1761853  109723    36   2.24  
         nothing   terribly_interesting    75.76  1761853   98788    66   3.70  
                   terribly_surprising     69.86  1761853   98788    67   3.76  
positive something terribly_wrong       4,487.29  1761853  318343  1707 308.43  
         all       terribly_wrong        -375.53  1761853  249279  1707 241.52  
         or        terribly_wrong        -560.11  1761853  326614  1707 316.45

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='definitely', axis=0), 'LRC', 10)

LRC        quant    f  unexp_f  dP1  \
polarity l1        l2                                                          
positive something definitely_wrong     4.65  existential  359   293.23 0.81   
         some      definitely_better    2.59  existential   43    38.63 0.58   
         something definitely_different 1.05  existential   34    26.59 0.65   

                                         dP2  dP1_simple  dP2_simple       G2  \
polarity l1        l2                                                           
positive something definitely_wrong     0.00        0.99        0.00 1,178.01   
         some      definitely_better    0.00        0.64        0.00   150.60   
         something definitely_different 0.00        0.83        0.00    81.66   

                                               N      f1   f2  exp_f  
polarity l1        l2                                                 
positive something definitely_wrong      1761853  318343  364  65.77  
         some      definitely_better     1761853  114929   67   4.37  
         something definitely_different  1761853  318343   41   7.41

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='slightly', axis=0), 'LRC', 10)

LRC        quant    f  unexp_f  dP1  \
polarity l1        l2                                                        
positive or        slightly_higher    3.24  existential  310   240.11 0.64   
                   slightly_more      3.04  existential  181   141.33 0.66   
                   slightly_lower     2.80  existential  165   127.55 0.63   
                   slightly_better    2.26  existential  190   139.58 0.51   
         something slightly_different 2.25  existential  743   516.24 0.41   
         or        slightly_less      2.00  existential   79    60.28 0.60   
                   slightly_larger    1.79  existential  143   101.10 0.45   
                   slightly_damp      1.61  existential   49    38.80 0.71   
                   slightly_wider     1.46  existential   55    44.25 0.76   
                   slightly_golden    1.40  existential   71    57.47 0.79   

                                       dP2  dP1_simple  dP2_simple       G2  \
polarity l1        l2                                                         
positive or        slightly_higher    0.00        0.82        0.00   719.79   
                   slightly_more      0.00        0.85        0.00   439.69   
                   slightly_lower     0.00        0.82        0.00   379.03   
                   slightly_better    0.00        0.70        0.00   341.14   
         something slightly_different 0.00        0.59        0.00 1,050.59   
         or        slightly_less      0.00        0.78        0.00   169.44   
                   slightly_larger    0.00        0.63        0.00   218.90   
                   slightly_damp      0.00        0.89        0.00   129.72   
                   slightly_wider     0.00        0.95        0.00   163.01   
                   slightly_golden    0.00        0.97        0.00   221.82   

                                             N      f1    f2  exp_f  
polarity l1        l2                                                
positive or        slightly_higher     1761853  326614   377  69.89  
                   slightly_more       1761853  326614   214  39.67  
                   slightly_lower      1761853  326614   202  37.45  
                   slightly_better     1761853  326614   272  50.42  
         something slightly_different  1761853  318343  1255 226.76  
         or        slightly_less       1761853  326614   101  18.72  
                   slightly_larger     1761853  326614   226  41.90  
                   slightly_damp       1761853  326614    55  10.20  
                   slightly_wider      1761853  326614    58  10.75  
                   slightly_golden     1761853  326614    73  13.53

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='highly', axis=0), 'LRC', 10)

LRC             quant    f  unexp_f  dP1  \
polarity l1    l2                                                             
positive both  highly_functional  2.53         universal   18    17.09 0.53   
               highly_efficient   2.46         universal   20    18.84 0.46   
         often highly_complex     2.30  hedged_universal   55    48.30 0.38   
         all   highly_qualified   2.24         universal  145   112.32 0.49   
         both  highly_intelligent 2.14         universal   33    29.82 0.27   
               highly_contagious  2.11         universal   17    15.92 0.42   
               highly_effective   2.07         universal   62    53.88 0.19   
               highly_addictive   2.00         universal   19    17.58 0.35   
               highly_successful  1.88         universal   35    30.94 0.22   
         many  highly_critical    1.74  hedged_universal   41    35.46 0.17   

                                   dP2  dP1_simple  dP2_simple     G2  \
polarity l1    l2                                                       
positive both  highly_functional  0.00        0.56        0.00  85.18   
               highly_efficient   0.00        0.49        0.00  86.88   
         often highly_complex     0.00        0.43        0.00 157.45   
         all   highly_qualified   0.00        0.63        0.00 288.41   
         both  highly_intelligent 0.00        0.29        0.00 103.86   
               highly_contagious  0.00        0.45        0.00  70.06   
               highly_effective   0.00        0.22        0.00 155.60   
               highly_addictive   0.00        0.38        0.00  70.74   
               highly_successful  0.00        0.24        0.00  96.42   
         many  highly_critical    0.00        0.20        0.00  99.91   

                                         N      f1   f2  exp_f  
polarity l1    l2                                               
positive both  highly_functional   1761853   50007   32   0.91  
               highly_efficient    1761853   50007   41   1.16  
         often highly_complex      1761853   92233  128   6.70  
         all   highly_qualified    1761853  249279  231  32.68  
         both  highly_intelligent  1761853   50007  112   3.18  
               highly_contagious   1761853   50007   38   1.08  
               highly_effective    1761853   50007  286   8.12  
               highly_addictive    1761853   50007   50   1.42  
               highly_successful   1761853   50007  143   4.06  
         many  highly_critical     1761853   47133  207   5.54

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='remotely', axis=0), 'LRC', 10)

LRC      quant    f  unexp_f  dP1  \
polarity l1         l2                                                        
negative none       remotely_true        4.66  not_exist   44    42.84 0.65   
         nothing    remotely_comparable  3.26  not_exist   42    39.36 0.84   
         none       remotely_possible    3.24  not_exist   23    22.12 0.44   
         nothing    remotely_close       3.22  not_exist  139   124.25 0.47   
                    remotely_similar     3.05  not_exist   64    58.06 0.55   
         none       remotely_accurate    2.51  not_exist   11    10.72 0.67   
         nothing    remotely_funny       2.48  not_exist   32    29.20 0.58   
                    remotely_interesting 2.41  not_exist   42    37.57 0.48   
                    remotely_new         1.66  not_exist   17    15.77 0.72   
         none       remotely_close       1.54  not_exist   34    29.37 0.11   
positive everyone   remotely_interested  1.61  universal   16    14.68 0.20   
         everything remotely_related     1.08  universal   15    13.32 0.18   

                                          dP2  dP1_simple  dP2_simple     G2  \
polarity l1         l2                                                         
negative none       remotely_true        0.00        0.67        0.00 272.29   
         nothing    remotely_comparable  0.00        0.89        0.00 210.75   
         none       remotely_possible    0.00        0.46        0.00 117.80   
         nothing    remotely_close       0.00        0.53        0.00 451.70   
                    remotely_similar     0.00        0.60        0.00 231.32   
         none       remotely_accurate    0.00        0.69        0.00  69.17   
         nothing    remotely_funny       0.00        0.64        0.00 121.14   
                    remotely_interesting 0.00        0.53        0.00 137.10   
                    remotely_new         0.00        0.77        0.00  74.96   
         none       remotely_close       0.00        0.13        0.00  80.33   
positive everyone   remotely_interested  0.00        0.22        0.00  53.75   
         everything remotely_related     0.00        0.20        0.00  41.59   

                                                N     f1   f2  exp_f  
polarity l1         l2                                                
negative none       remotely_true         1761853  31021   66   1.16  
         nothing    remotely_comparable   1761853  98788   47   2.64  
         none       remotely_possible     1761853  31021   50   0.88  
         nothing    remotely_close        1761853  98788  263  14.75  
                    remotely_similar      1761853  98788  106   5.94  
         none       remotely_accurate     1761853  31021   16   0.28  
         nothing    remotely_funny        1761853  98788   50   2.80  
                    remotely_interesting  1761853  98788   79   4.43  
                    remotely_new          1761853  98788   22   1.23  
         none       remotely_close        1761853  31021  263   4.63  
positive everyone   remotely_interested   1761853  32327   72   1.32  
         everything remotely_related      1761853  39516   75   1.68

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='absolutely', axis=0), 'LRC', 10)

LRC        quant    f  unexp_f  dP1  \
polarity l1         l2                                                          
negative nobody     absolutely_sure      2.68    not_exist   13    12.57 0.11   
         never      absolutely_sure      2.23    not_exist   56    48.59 0.41   
                    absolutely_certain   2.20    not_exist   67    57.41 0.37   
positive everything absolutely_perfect   4.75    universal  120   115.36 0.56   
                    absolutely_true      3.60    universal   73    68.87 0.37   
                    absolutely_ideal     2.93    universal   16    15.51 0.70   
                    absolutely_delicious 2.92    universal   44    41.02 0.31   
                    absolutely_free      2.91    universal   84    76.82 0.24   
                    absolutely_fine      2.53    universal   23    21.61 0.35   
         something  absolutely_new       2.19  existential   83    64.39 0.63   
         everything absolutely_beautiful 2.08    universal   28    25.51 0.23   
         everyone   absolutely_delighted 1.98    universal   17    15.84 0.25   
         everything absolutely_normal    1.76    universal   12    11.30 0.36   

                                          dP2  dP1_simple  dP2_simple     G2  \
polarity l1         l2                                                         
negative nobody     absolutely_sure      0.00        0.11        0.00  65.12   
         never      absolutely_sure      0.00        0.47        0.00 154.50   
                    absolutely_certain   0.00        0.44        0.00 172.34   
positive everything absolutely_perfect   0.00        0.58        0.00 633.99   
                    absolutely_true      0.00        0.40        0.00 312.41   
                    absolutely_ideal     0.00        0.73        0.00  96.01   
                    absolutely_delicious 0.00        0.33        0.00 169.41   
                    absolutely_free      0.00        0.26        0.00 280.41   
                    absolutely_fine      0.00        0.37        0.00  94.69   
         something  absolutely_new       0.00        0.81        0.00 190.61   
         everything absolutely_beautiful 0.00        0.25        0.00  91.05   
         everyone   absolutely_delighted 0.00        0.27        0.00  64.18   
         everything absolutely_normal    0.00        0.39        0.00  50.62   

                                                N      f1   f2  exp_f  
polarity l1         l2                                                 
negative nobody     absolutely_sure       1761853    6311  119   0.43  
         never      absolutely_sure       1761853  109723  119   7.41  
                    absolutely_certain    1761853  109723  154   9.59  
positive everything absolutely_perfect    1761853   39516  207   4.64  
                    absolutely_true       1761853   39516  184   4.13  
                    absolutely_ideal      1761853   39516   22   0.49  
                    absolutely_delicious  1761853   39516  133   2.98  
                    absolutely_free       1761853   39516  320   7.18  
                    absolutely_fine       1761853   39516   62   1.39  
         something  absolutely_new        1761853  318343  103  18.61  
         everything absolutely_beautiful  1761853   39516  111   2.49  
         everyone   absolutely_delighted  1761853   32327   63   1.16  
         everything absolutely_normal     1761853   39516   31   0.70

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='downright', axis=0), 'LRC', 10)

LRC        quant   f  unexp_f  dP1  \
polarity l1        l2                                                          
positive sometimes downright_scary       2.03  existential  40    35.25 0.31   
                   downright_difficult   1.97  existential  16    15.11 0.72   
                   downright_painful     1.72  existential  24    21.41 0.35   
                   downright_frustrating 1.71  existential  14    13.19 0.69   
         some      downright_awful       1.59  existential  34    28.91 0.37   
                   downright_ugly        1.54  existential  33    27.98 0.36   
         sometimes downright_bizarre     1.40  existential  30    25.55 0.24   
         some      downright_dangerous   1.40  existential  51    40.82 0.26   
                   downright_hilarious   1.36  existential  33    27.46 0.32   
         or        downright_impossible  1.36  existential  57    41.80 0.51   

                                          dP2  dP1_simple  dP2_simple     G2  \
polarity l1        l2                                                          
positive sometimes downright_scary       0.00        0.36        0.00 113.11   
                   downright_difficult   0.00        0.76        0.00  78.52   
                   downright_painful     0.00        0.39        0.00  73.15   
                   downright_frustrating 0.00        0.74        0.00  67.03   
         some      downright_awful       0.00        0.44        0.00  84.73   
                   downright_ugly        0.00        0.43        0.00  80.94   
         sometimes downright_bizarre     0.00        0.29        0.00  70.50   
         some      downright_dangerous   0.00        0.33        0.00  95.44   
                   downright_hilarious   0.00        0.39        0.00  73.64   
         or        downright_impossible  0.00        0.70        0.00 101.54   

                                                N      f1   f2  exp_f  
polarity l1        l2                                                  
positive sometimes downright_scary        1761853   74707  112   4.75  
                   downright_difficult    1761853   74707   21   0.89  
                   downright_painful      1761853   74707   61   2.59  
                   downright_frustrating  1761853   74707   19   0.81  
         some      downright_awful        1761853  114929   78   5.09  
                   downright_ugly         1761853  114929   77   5.02  
         sometimes downright_bizarre      1761853   74707  105   4.45  
         some      downright_dangerous    1761853  114929  156  10.18  
                   downright_hilarious    1761853  114929   85   5.54  
         or        downright_impossible   1761853  326614   82  15.20

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='utterly', axis=0), 'LRC', 10)

LRC        quant   f  unexp_f  dP1  dP2  \
polarity l1        l2                                                           
positive something utterly_different 1.89  existential  63    48.91 0.63 0.00   
                   utterly_unique    1.08  existential  37    28.15 0.57 0.00   

                                      dP1_simple  dP2_simple     G2        N  \
polarity l1        l2                                                          
positive something utterly_different        0.81        0.00 145.20  1761853   
                   utterly_unique           0.76        0.00  76.85  1761853   

                                          f1  f2  exp_f  
polarity l1        l2                                    
positive something utterly_different  318343  78  14.09  
                   utterly_unique     318343  49   8.85

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='slightly', axis=0), 'LRC', 10)

LRC        quant    f  unexp_f  dP1  \
polarity l1        l2                                                        
positive or        slightly_higher    3.24  existential  310   240.11 0.64   
                   slightly_more      3.04  existential  181   141.33 0.66   
                   slightly_lower     2.80  existential  165   127.55 0.63   
                   slightly_better    2.26  existential  190   139.58 0.51   
         something slightly_different 2.25  existential  743   516.24 0.41   
         or        slightly_less      2.00  existential   79    60.28 0.60   
                   slightly_larger    1.79  existential  143   101.10 0.45   
                   slightly_damp      1.61  existential   49    38.80 0.71   
                   slightly_wider     1.46  existential   55    44.25 0.76   
                   slightly_golden    1.40  existential   71    57.47 0.79   

                                       dP2  dP1_simple  dP2_simple       G2  \
polarity l1        l2                                                         
positive or        slightly_higher    0.00        0.82        0.00   719.79   
                   slightly_more      0.00        0.85        0.00   439.69   
                   slightly_lower     0.00        0.82        0.00   379.03   
                   slightly_better    0.00        0.70        0.00   341.14   
         something slightly_different 0.00        0.59        0.00 1,050.59   
         or        slightly_less      0.00        0.78        0.00   169.44   
                   slightly_larger    0.00        0.63        0.00   218.90   
                   slightly_damp      0.00        0.89        0.00   129.72   
                   slightly_wider     0.00        0.95        0.00   163.01   
                   slightly_golden    0.00        0.97        0.00   221.82   

                                             N      f1    f2  exp_f  
polarity l1        l2                                                
positive or        slightly_higher     1761853  326614   377  69.89  
                   slightly_more       1761853  326614   214  39.67  
                   slightly_lower      1761853  326614   202  37.45  
                   slightly_better     1761853  326614   272  50.42  
         something slightly_different  1761853  318343  1255 226.76  
         or        slightly_less       1761853  326614   101  18.72  
                   slightly_larger     1761853  326614   226  41.90  
                   slightly_damp       1761853  326614    55  10.20  
                   slightly_wider      1761853  326614    58  10.75  
                   slightly_golden     1761853  326614    73  13.53

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='extremely', axis=0), 'LRC', 10)

LRC             quant    f  unexp_f  \
polarity l1       l2                                                          
positive all      extremely_innovative 5.23         universal  379   324.24   
         everyone extremely_friendly   4.78         universal  103    99.50   
                  extremely_nice       4.54         universal   65    62.85   
                  extremely_welcoming  3.41         universal   24    23.14   
         both     extremely_talented   2.87         universal   55    50.63   
         always   extremely_careful    2.80         universal   39    35.87   
         many     extremely_scarce     2.68  hedged_universal   17    16.28   
         everyone extremely_supportive 2.57         universal   32    29.78   
                  extremely_excited    2.57         universal   50    45.74   
         always   extremely_responsive 2.57         universal   25    23.32   

                                        dP1  dP2  dP1_simple  dP2_simple  \
polarity l1       l2                                                       
positive all      extremely_innovative 0.84 0.00        0.98        0.00   
         everyone extremely_friendly   0.52 0.00        0.54        0.00   
                  extremely_nice       0.54 0.00        0.56        0.00   
                  extremely_welcoming  0.49 0.00        0.51        0.00   
         both     extremely_talented   0.33 0.00        0.36        0.00   
         always   extremely_careful    0.54 0.00        0.58        0.00   
         many     extremely_scarce     0.60 0.00        0.63        0.00   
         everyone extremely_supportive 0.25 0.00        0.26        0.00   
                  extremely_excited    0.20 0.00        0.22        0.00   
         always   extremely_responsive 0.65 0.00        0.69        0.00   

                                             G2        N      f1   f2  exp_f  
polarity l1       l2                                                          
positive all      extremely_innovative 1,407.34  1761853  249279  387  54.76  
         everyone extremely_friendly     563.60  1761853   32327  191   3.50  
                  extremely_nice         361.07  1761853   32327  117   2.15  
                  extremely_welcoming    127.65  1761853   32327   47   0.86  
         both     extremely_talented     196.83  1761853   50007  154   4.37  
         always   extremely_careful      150.69  1761853   82200   67   3.13  
         many     extremely_scarce        88.07  1761853   47133   27   0.72  
         everyone extremely_supportive   119.41  1761853   32327  121   2.22  
                  extremely_excited      164.80  1761853   32327  232   4.26  
         always   extremely_responsive   109.99  1761853   82200   36   1.68

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='necessarily', axis=0), 'LRC', 10)

LRC      quant    f  unexp_f  dP1  \
polarity l1      l2                                                         
negative neither necessarily_true      6.22  not_exist   32    31.79 0.55   
         nothing necessarily_wrong     5.42  not_exist  194   182.00 0.85   
         neither necessarily_better    3.95  not_exist   10     9.89 0.34   
         none    necessarily_bad       3.35  not_exist   25    24.05 0.45   
         neither necessarily_bad       3.35  not_exist   11    10.80 0.20   
         none    necessarily_better    1.95  not_exist   11    10.49 0.36   
         nor     necessarily_desirable 1.85  not_exist   12    11.89 0.99   
         none    necessarily_true      1.20  not_exist   12    10.98 0.19   

                                        dP2  dP1_simple  dP2_simple     G2  \
polarity l1      l2                                                          
negative neither necessarily_true      0.00        0.55        0.00 279.53   
         nothing necessarily_wrong     0.00        0.91        0.00 987.67   
         neither necessarily_better    0.00        0.34        0.00  74.97   
         none    necessarily_bad       0.00        0.46        0.00 128.46   
         neither necessarily_bad       0.00        0.20        0.00  69.13   
         none    necessarily_better    0.00        0.38        0.00  51.02   
         nor     necessarily_desirable 0.00        1.00        0.00 113.54   
         none    necessarily_true      0.00        0.21        0.00  39.45   

                                              N     f1   f2  exp_f  
polarity l1      l2                                                 
negative neither necessarily_true       1761853   6458   58   0.21  
         nothing necessarily_wrong      1761853  98788  214  12.00  
         neither necessarily_better     1761853   6458   29   0.11  
         none    necessarily_bad        1761853  31021   54   0.95  
         neither necessarily_bad        1761853   6458   54   0.20  
         none    necessarily_better     1761853  31021   29   0.51  
         nor     necessarily_desirable  1761853  15546   12   0.11  
         none    necessarily_true       1761853  31021   58   1.02

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='quite', axis=0), 'LRC', 15)

LRC             quant     f  unexp_f  \
polarity l1        l2                                                          
negative nobody    quite_sure          5.96         not_exist   660   649.31   
         never     quite_able          5.76         not_exist   319   297.27   
                   quite_sure          5.00         not_exist  2176 1,990.17   
                   quite_clear         4.41         not_exist   399   363.75   
                   quite_enough        4.36         not_exist   150   138.17   
         neither   quite_ready         4.27         not_exist    28    27.46   
         never     quite_satisfied     4.15         not_exist   144   132.11   
         nobody    quite_certain       3.48         not_exist    23    22.34   
         never     quite_certain       3.44         not_exist   117   105.48   
                   quite_right         3.24         not_exist   217   191.16   
         none      quite_right         2.95         not_exist    89    81.69   
         never     quite_complete      2.42         not_exist    31    28.20   
         nobody    quite_ready         2.36         not_exist    13    12.47   
         never     quite_comfortable   2.34         not_exist    67    58.03   
                   quite_ready         1.94         not_exist    59    49.78   
positive all       quite_innovative    4.87         universal   370   314.54   
         something quite_special       4.38       existential   404   325.58   
                   quite_different     3.20       existential  2268 1,697.03   
         some      quite_large         3.01       existential   222   192.45   
         something quite_extraordinary 2.88       existential   164   128.04   
         some      quite_old           2.83       existential   110    96.24   
         something quite_magical       2.70       existential    95    76.03   
         sometimes quite_difficult     2.59       existential   170   146.85   
         something quite_unique        2.58       existential   224   168.89   
         some      quite_elaborate     2.52       existential    45    40.17   
         often     quite_poor          2.42  hedged_universal    35    31.70   
                   quite_difficult     2.35  hedged_universal   176   147.42   
         some      quite_sophisticated 2.29       existential    32    28.80   
                   quite_graphic       2.27       existential    31    27.93   
         something quite_remarkable    2.26       existential   146   109.32   

                                        dP1  dP2  dP1_simple  dP2_simple  \
polarity l1        l2                                                      
negative nobody    quite_sure          0.22 0.10        0.22        0.10   
         never     quite_able          0.85 0.00        0.91        0.00   
                   quite_sure          0.67 0.02        0.73        0.02   
                   quite_clear         0.64 0.00        0.70        0.00   
                   quite_enough        0.73 0.00        0.79        0.00   
         neither   quite_ready         0.19 0.00        0.19        0.00   
         never     quite_satisfied     0.69 0.00        0.75        0.00   
         nobody    quite_certain       0.12 0.00        0.12        0.00   
         never     quite_certain       0.57 0.00        0.63        0.00   
                   quite_right         0.46 0.00        0.52        0.00   
         none      quite_right         0.20 0.00        0.21        0.00   
         never     quite_complete      0.63 0.00        0.69        0.00   
         nobody    quite_ready         0.08 0.00        0.09        0.00   
         never     quite_comfortable   0.40 0.00        0.47        0.00   
                   quite_ready         0.34 0.00        0.40        0.00   
positive all       quite_innovative    0.80 0.00        0.94        0.00   
         something quite_special       0.75 0.00        0.93        0.00   
                   quite_different 

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='terribly', axis=0), 'LRC', 15)

LRC        quant     f  unexp_f  \
polarity l1        l2                                                       
negative nothing   terribly_new          4.00    not_exist    64    60.02   
         none      terribly_surprising   3.67    not_exist    32    30.82   
         nothing   terribly_exciting     3.30    not_exist    71    64.83   
                   terribly_original     3.13    not_exist    38    35.42   
         none      terribly_interesting  2.26    not_exist    19    17.84   
         nothing   terribly_unusual      1.90    not_exist    20    18.65   
         never     terribly_interested   1.73    not_exist    24    21.32   
                   terribly_good         1.62    not_exist    21    18.76   
         nothing   terribly_interesting  1.59    not_exist    28    24.30   
                   terribly_surprising   1.46    not_exist    27    23.24   
         never     terribly_fond         1.26    not_exist    15    13.75   
                   terribly_popular      1.20    not_exist    15    13.82   
         nothing   terribly_complicated  1.11    not_exist    14    12.60   
                   terribly_serious      1.04    not_exist    13    11.77   
positive something terribly_wrong        4.98  existential  1571 1,262.57   
         all       terribly_wrong       -1.95    universal    20  -221.52   
         or        terribly_wrong       -2.50  existential    17  -299.45   

                                          dP1   dP2  dP1_simple  dP2_simple  \
polarity l1        l2                                                         
negative nothing   terribly_new          0.85  0.00        0.90        0.00   
         none      terribly_surprising   0.46  0.00        0.48        0.00   
         nothing   terribly_exciting     0.59  0.00        0.65        0.00   
                   terribly_original     0.77  0.00        0.83        0.00   
         none      terribly_interesting  0.27  0.00        0.29        0.00   
         nothing   terribly_unusual      0.78  0.00        0.83        0.00   
         never     terribly_interested   0.50  0.00        0.56        0.00   
                   terribly_good         0.52  0.00        0.58        0.00   
         nothing   terribly_interesting  0.37  0.00        0.42        0.00   
                   terribly_surprising   0.35  0.00        0.40        0.00   
         never     terribly_fond         0.69  0.00        0.75        0.00   
                   terribly_popular      0.73  0.00        0.79        0.00   
         nothing   terribly_complicated  0.50  0.00        0.56        0.00   
                   terribly_serious      0.53  0.00        0.59        0.00   
positive something terribly_wrong        0.74  0.00        0.92        0.00   
         all       terribly_wrong       -0.13 -0.00        0.01        0.00   
         or        terribly_wrong       -0.18 -0.00        0.01        0.00   

                                              G2        N      f1    f2  exp_f  
polarity l1        l2                                                           
negative nothing   terribly_new           323.91  1761853   98788    71   3.98  
         none      terribly_surprising    167.05  1761853   31021    67   1.18  
         nothing   terribly_exciting      270.62  1761853   98788   110   6.17  
                   terribly_original      177.40  1761853   98788    46   2.58  
         none      terribly_interesting    75.95  1761853   31021    66   1.16  
         nothing   terribly_unusual        94.08  1761853   98788    24   1.35  
         never     terribly_interested     76.68  1761853  109723    43   2.68  
                   terribly_good           69.63  1761853  109723    36   2.24  
         nothing   terribly_interesting    75.76  1761853   98788    66   3.70  
                   terribly_surprising     69.86  1761853   98788    67   3.76  
         never     terribly_fond           61.44  1761853  109723    20   1.25  
                   terr

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='dreadfully', axis=0), 'LRC', 15)

,,,LRC,quant,f,unexp_f,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f
polarity,l1,l2,,,,,,,,,,,,,
positive,something,dreadfully_wrong,2.69,existential,97,78.57,0.77,0.00,0.95,0.00,294.04,1761853,318343,102,18.43


In [ ]:
show_metric_top(sig_trig_abbr.filter(like='particularly', axis=0), 'LRC', 15)

LRC      quant    f  unexp_f  dP1  \
polarity l1      l2                                                           
negative nothing particularly_wrong      5.83  not_exist  211   198.78 0.91   
                 particularly_new        5.20  not_exist  336   313.01 0.76   
                 particularly_special    4.85  not_exist  318   294.79 0.71   
                 particularly_unusual    4.70  not_exist  152   141.63 0.77   
                 particularly_remarkable 4.47  not_exist   96    89.78 0.81   
         neither particularly_effective  4.41  not_exist   24    23.61 0.22   
         none    particularly_surprising 4.33  not_exist   78    75.04 0.45   
         neither particularly_appealing  3.91  not_exist   16    15.73 0.21   
                 particularly_good       3.84  not_exist   52    50.24 0.10   
         none    particularly_likeable   3.61  not_exist   19    18.54 0.71   
         neither particularly_impressive 3.61  not_exist   18    17.59 0.16   
                 particularly_easy       3.51  not_exist   17    16.61 0.16   
         none    particularly_difficult  3.49  not_exist   64    60.51 0.31   
         neither particularly_attractive 3.49  not_exist   12    11.79 0.20   
         nothing particularly_unique     3.48  not_exist   82    75.05 0.61   

                                          dP2  dP1_simple  dP2_simple  \
polarity l1      l2                                                     
negative nothing particularly_wrong      0.00        0.97        0.00   
                 particularly_new        0.00        0.82        0.00   
                 particularly_special    0.00        0.77        0.00   
                 particularly_unusual    0.00        0.82        0.00   
                 particularly_remarkable 0.00        0.86        0.00   
         neither particularly_effective  0.00        0.23        0.00   
         none    particularly_surprising 0.00        0.46        0.00   
         neither particularly_appealing  0.00        0.22        0.00   
                 particularly_good       0.01        0.11        0.01   
         none    particularly_likeable   0.00        0.73        0.00   
         neither particularly_impressive 0.00        0.16        0.00   
                 particularly_easy       0.00        0.16        0.00   
         none    particularly_difficult  0.00        0.32        0.00   
         neither particularly_attractive 0.00        0.21        0.00   
         nothing particularly_unique     0.00        0.66        0.00   

                                               G2        N     f1   f2  exp_f  
polarity l1      l2                                                            
negative nothing particularly_wrong      1,155.16  1761853  98788  218  12.22  
                 particularly_new        1,558.57  1761853  98788  410  22.99  
                 particularly_special    1,396.02  1761853  98788  414  23.21  
                 particularly_unusual      706.39  1761853  98788  185  10.37  
                 particularly_remarkable   467.08  1761853  98788  111   6.22  
         neither particularly_effective    156.51  1761853   6458  106   0.39  
         none    particularly_surprising   401.50  1761853  31021  168   2.96  
         neither particularly_appealing    102.68  1761853   6458   74   0.27  
                 particularly_good         257.56  1761853   6458  480   1.76  
         none    particularly_likeable     123.47  1761853  31021   26   0.46  
         neither particularly_impressive   104.25  1761853   6458  111   0.41  
                 particularly_easy          97.71  1761853   6458  107   0.39  
         none    particularly_difficult    272.74  1761853  31021  198   3.49  
         neither particularly_attractive    75.83  1761853   6458   58   0.21  
         nothing particularly_unique       318.65  1761853  98788  124   6.95

In [ ]:
show_metric_top(ex_trig_abbr.filter(like='inherently', axis=0), 'LRC', 15)

LRC        quant     f  unexp_f  \
polarity l1        l2                                                          
negative nothing   inherently_wrong         5.74    not_exist  1472 1,371.13   
         neither   inherently_better        3.66    not_exist    13    12.78   
                   inherently_good          2.91    not_exist    15    14.53   
         nothing   inherently_bad           2.65    not_exist    86    75.57   
                   inherently_evil          2.49    not_exist    51    45.39   
         none      inherently_bad           2.35    not_exist    38    34.73   
         nothing   inherently_special       2.25    not_exist    25    22.93   
                   inherently_good          1.93    not_exist    50    42.77   
                   inherently_immoral       1.67    not_exist    17    15.65   
                   inherently_dangerous     1.31    not_exist    29    24.40   
                   inherently_difficult     0.85    not_exist    13    11.54   
                   inherently_improper      0.80    not_exist    15    14.05   
         none      inherently_better        0.64    not_exist    10     8.94   
         nothing   inherently_contradictory 0.57    not_exist    10     9.10   
         nor       inherently_bad           0.50    not_exist    12    10.36   
positive something inherently_funny         0.53  existential    34    23.70   
                   inherently_comforting    0.00  existential    11     9.01   
         all       inherently_good          0.00    universal    13    -5.25   
         or        inherently_bad           0.00  existential    10   -24.48   
         something inherently_bad           0.00  existential    20   -13.61   
         or        inherently_evil          0.00  existential    12    -6.54   
                   inherently_dangerous     0.00  existential    15    -0.20   
         something inherently_evil          0.00  existential    16    -2.07   
                   inherently_good          0.00  existential    20    -3.31   
                   inherently_wrong         0.00  existential   301   -24.05   
                   inherently_unfair        0.00  existential    14     9.66   
                   inherently_special       0.00  existential    12     5.31   
                   inherently_flawed        0.00  existential    11     5.58   
                   inherently_valuable      0.00  existential    10     6.57   
                   inherently_different     0.00  existential    25    16.51   

                                              dP1   dP2  dP1_simple  \
polarity l1        l2                                                 
negative nothing   inherently_wrong          0.76  0.01        0.82   
         neither   inherently_better         0.21  0.00        0.22   
                   inherently_good           0.11  0.00        0.12   
         nothing   inherently_bad            0.41  0.00        0.46   
                   inherently_evil           0.45  0.00        0.51   
         none      inherently_bad            0.19  0.00        0.20   
         nothing   inherently_special        0.62  0.00        0.68   
                   inherently_good           0.33  0.00        0.39   
                   inherently_immoral        0.65  0.00        0.71   
                   inherently_dangerous      0.30  0.00        0.35   
                   inherently_difficult      0.44  0.00        0.50   
                   inherently_improper       0.83  0.00        0.88   
         none      inherently_better         0.15  0.00        0.17   
         nothing   inherently_contradictory  0.57  0.00        0.62   
         nor       inherently_bad            0.06  0.00        0.06   
positive something inherently_funny          0.42  0.00        0.60   
                   inherently_comforting     0.82  0.00        1.00   
         all       inherently_good          -0.04 -0.00        0.10   
         or        inherently_bad           -0.13 -0.00        0

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='terribly', axis=0), 'dP1', 15)

dP1        quant     f  unexp_f  \
polarity l1        l2                                                       
negative nothing   terribly_new          0.85    not_exist    64    60.02   
                   terribly_unusual      0.78    not_exist    20    18.65   
                   terribly_original     0.77    not_exist    38    35.42   
         never     terribly_popular      0.73    not_exist    15    13.82   
                   terribly_fond         0.69    not_exist    15    13.75   
         nothing   terribly_exciting     0.59    not_exist    71    64.83   
                   terribly_serious      0.53    not_exist    13    11.77   
         never     terribly_good         0.52    not_exist    21    18.76   
         nothing   terribly_complicated  0.50    not_exist    14    12.60   
         never     terribly_interested   0.50    not_exist    24    21.32   
         none      terribly_surprising   0.46    not_exist    32    30.82   
         nothing   terribly_interesting  0.37    not_exist    28    24.30   
                   terribly_surprising   0.35    not_exist    27    23.24   
         none      terribly_interesting  0.27    not_exist    19    17.84   
positive something terribly_wrong        0.74  existential  1571 1,262.57   
         all       terribly_wrong       -0.13    universal    20  -221.52   
         or        terribly_wrong       -0.18  existential    17  -299.45   

                                          LRC   dP2  dP1_simple  dP2_simple  \
polarity l1        l2                                                         
negative nothing   terribly_new          4.00  0.00        0.90        0.00   
                   terribly_unusual      1.90  0.00        0.83        0.00   
                   terribly_original     3.13  0.00        0.83        0.00   
         never     terribly_popular      1.20  0.00        0.79        0.00   
                   terribly_fond         1.26  0.00        0.75        0.00   
         nothing   terribly_exciting     3.30  0.00        0.65        0.00   
                   terribly_serious      1.04  0.00        0.59        0.00   
         never     terribly_good         1.62  0.00        0.58        0.00   
         nothing   terribly_complicated  1.11  0.00        0.56        0.00   
         never     terribly_interested   1.73  0.00        0.56        0.00   
         none      terribly_surprising   3.67  0.00        0.48        0.00   
         nothing   terribly_interesting  1.59  0.00        0.42        0.00   
                   terribly_surprising   1.46  0.00        0.40        0.00   
         none      terribly_interesting  2.26  0.00        0.29        0.00   
positive something terribly_wrong        4.98  0.00        0.92        0.00   
         all       terribly_wrong       -1.95 -0.00        0.01        0.00   
         or        terribly_wrong       -2.50 -0.00        0.01        0.00   

                                              G2        N      f1    f2  exp_f  
polarity l1        l2                                                           
negative nothing   terribly_new           323.91  1761853   98788    71   3.98  
                   terribly_unusual        94.08  1761853   98788    24   1.35  
                   terribly_original      177.40  1761853   98788    46   2.58  
         never     terribly_popular        64.24  1761853  109723    19   1.18  
                   terribly_fond           61.44  1761853  109723    20   1.25  
         nothing   terribly_exciting      270.62  1761853   98788   110   6.17  
                   terribly_serious        46.18  1761853   98788    22   1.23  
         never     terribly_good           69.63  1761853  109723    36   2.24  
         nothing   terribly_complicated    47.65  1761853   98788    25   1.40  
         never     terribly_interested     76.68  1761853  109723    43   2.68  
         none      terribly_surprising    167.05  1761853   31021    67   1.18  
         nothing   terr

In [ ]:
show_metric_top(sig_trig_abbr.filter(regex='~un\w+_', axis=0), 'dP1', 15)

dP1             quant   f  \
polarity l1        l2                                                     
positive something uniquely_special          0.70       existential  58   
                   uniquely_american         0.58       existential  38   
         many      understandably_concerned  0.41  hedged_universal  15   
         all       uniquely_different        0.39         universal  53   
         often     unintentionally_hilarious 0.38  hedged_universal  19   

                                              unexp_f  LRC  dP2  dP1_simple  \
polarity l1        l2                                                         
positive something uniquely_special             46.07 1.94 0.00        0.88   
                   uniquely_american            28.97 1.13 0.00        0.76   
         many      understandably_concerned     14.09 2.01 0.00        0.44   
         all       uniquely_different           38.85 1.13 0.00        0.53   
         often     unintentionally_hilarious    16.70 1.28 0.00        0.43   

                                              dP2_simple     G2        N  \
polarity l1        l2                                                      
positive something uniquely_special                 0.00 152.92  1761853   
                   uniquely_american                0.00  79.71  1761853   
         many      understandably_concerned         0.00  63.01  1761853   
         all       uniquely_different               0.00  83.37  1761853   
         often     unintentionally_hilarious        0.00  54.61  1761853   

                                                  f1   f2  exp_f  
polarity l1        l2                                             
positive something uniquely_special           318343   66  11.93  
                   uniquely_american          318343   50   9.03  
         many      understandably_concerned    47133   34   0.91  
         all       uniquely_different         249279  100  14.15  
         often     unintentionally_hilarious   92233   44   2.30

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='_un', axis=0), 'dP1', 15)

dP1             quant    f  unexp_f  \
polarity l1        l2                                                           
negative nothing   more_unattractive     0.89         not_exist   32    30.09   
                   terribly_unusual      0.78         not_exist   20    18.65   
                   particularly_unusual  0.77         not_exist  152   141.63   
                   too_unusual           0.73         not_exist   92    85.44   
                   more_unsightly        0.67         not_exist   16    14.77   
                   too_unexpected        0.67         not_exist   13    11.99   
                   more_unappealing      0.64         not_exist   14    12.88   
                   more_unjust           0.64         not_exist   23    21.15   
                   especially_unusual    0.62         not_exist   17    15.60   
                   particularly_unique   0.61         not_exist   82    75.05   
                   especially_unique     0.44         not_exist   17    15.09   
                   more_unpleasant       0.35         not_exist   33    28.46   
                   more_uncomfortable    0.35         not_exist   68    58.64   
         never     more_uncertain        0.26         not_exist   36    28.96   
         none      as_unusual            0.20         not_exist   13    11.94   
positive many      also_unaware          0.97  hedged_universal   21    20.44   
                   still_unaware         0.85  hedged_universal   84    81.43   
         or        otherwise_unable      0.80       existential  343   278.49   
                   otherwise_unavailable 0.78       existential  160   129.41   
         something wholly_unique         0.77       existential   83    67.28   
                   almost_unheard        0.77       existential  142   115.08   
                   truly_unique          0.76       existential  965   779.80   
         many      previously_uninsured  0.76  hedged_universal   29    28.01   
         or        very_unfavorable      0.75       existential   45    36.10   
                   even_unsafe           0.75       existential   44    35.29   
                   just_unsure           0.75       existential   44    35.29   
         sometimes just_unavoidable      0.69       existential   14    13.19   
         or        even_unknown          0.68       existential   60    47.21   
         many      now_unable            0.67  hedged_universal   14    13.46   
         sometimes simply_unavoidable    0.67       existential   15    14.11   

                                          LRC  dP2  dP1_simple  dP2_simple  \
polarity l1        l2                                                        
negative nothing   more_unattractive     2.09 0.00        0.94        0.00   
                   terribly_unusual      1.90 0.00        0.83        0.00   
                   particularly_unusual  4.70 0.00        0.82        0.00   
                   too_unusual           4.10 0.00        0.79        0.00   
                   more_unsightly        1.56 0.00        0.73        0.00   
                   too_unexpected        1.13 0.00        0.72        0.00   
                   more_unappealing      1.29 0.00        0.70        0.00   
                   more_unjust           2.17 0.00        0.70        0.00   
                   especially_unusual    1.64 0.00        0.68        0.00   
                   particularly_unique   3.48 0.00        0.66        0.00   
                   especially_unique     1.26 0.00        0.50        0.00   
                   more_unpleasant       1.68 0.00        0.41        0.00   
                   more_uncomfortable    2.24 0.00        0.41        0.00   
         never     more_uncertain        1.16 0.00        0.32        0.00   
         none      as_unusual            1.38 0.00        0.22        0.00   
positive many      also_unaware          1.17 0.00        1.00        0.00   
                   still_unaware         5.46

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='_non', axis=0), 'dP1', 15)

dP1        quant    f  unexp_f  LRC  dP2  \
polarity l1 l2                                                            
positive or even_nonexistent  0.81  existential  144   117.31 1.07 0.00   
            even_non-existent 0.77  existential  201   162.07 3.82 0.00   

                               dP1_simple  dP2_simple     G2        N      f1  \
polarity l1 l2                                                                  
positive or even_nonexistent         1.00        0.00 485.43  1761853  326614   
            even_non-existent        0.96        0.00 606.99  1761853  326614   

                                f2  exp_f  
polarity l1 l2                             
positive or even_nonexistent   144  26.69  
            even_non-existent  210  38.93

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='_in', axis=0), 'dP1', 15)

dP1        quant    f  unexp_f  \
polarity l1         l2                                                        
negative nothing    more_invigorating       0.79    not_exist   51    47.64   
         nor        most_intelligent        0.74    not_exist   45    44.47   
         nothing    more_incorrect          0.71    not_exist   13    12.05   
         never      that_interested         0.68    not_exist   50    45.83   
         nothing    too_interesting         0.65    not_exist   71    65.39   
         never      overly_interested       0.65    not_exist   17    15.51   
         nothing    more_intoxicating       0.59    not_exist   13    11.88   
                    particularly_innovative 0.57    not_exist   34    30.97   
                    more_insulting          0.57    not_exist   35    31.86   
         none       more_infamous           0.56    not_exist   15    14.54   
         nothing    more_indicative         0.54    not_exist   51    46.23   
                    more_infuriating        0.53    not_exist   91    82.31   
         never      terribly_interested     0.50    not_exist   24    21.32   
         nothing    more_inconvenient       0.48    not_exist   21    18.81   
                    remotely_interesting    0.48    not_exist   42    37.57   
positive someone    as_inexperienced        0.85  existential   13    12.75   
         all        extremely_innovative    0.84    universal  379   324.24   
                    incredibly_innovative   0.83    universal  365   312.08   
         or         otherwise_inappropriate 0.81  existential  199   162.11   
         all        quite_innovative        0.80    universal  370   314.54   
         something  as_innocuous            0.78  existential  304   246.72   
         or         even_international      0.76  existential  155   124.60   
                    simply_interested       0.75  existential  326   261.49   
         everything so_instant              0.75    universal   10     9.71   
         or         just_interested         0.75  existential  723   578.96   
                    simply_incompetent      0.73  existential   46    36.73   
         all        very_innovative         0.73    universal  368   308.43   
         or         otherwise_inaccessible  0.73  existential   43    34.29   
         all        really_innovative       0.72    universal  382   319.46   
         always     so_informative          0.72    universal   13    12.21   

                                             LRC  dP2  dP1_simple  dP2_simple  \
polarity l1         l2                                                          
negative nothing    more_invigorating       3.61 0.00        0.85        0.00   
         nor        most_intelligent        5.95 0.00        0.75        0.00   
         nothing    more_incorrect          1.08 0.00        0.76        0.00   
         never      that_interested         3.16 0.00        0.75        0.00   
         nothing    too_interesting         3.56 0.00        0.71        0.00   
         never      overly_interested       1.51 0.00        0.71        0.00   
         nothing    more_intoxicating       1.12 0.00        0.65        0.00   
                    particularly_innovative 2.53 0.00        0.63        0.00   
                    more_insulting          2.54 0.00        0.62        0.00   
         none       more_infamous           2.99 0.00        0.58        0.00   
         nothing    more_indicative         2.84 0.00        0.60        0.00   
                    more_infuriating        3.24 0.00        0.59        0.00   
         never      terribly_interested     1.73 0.00        0.56        0.00   
         nothing    more_inconvenient       1.66 0.00        0.54        0.00   
                    remotely_interesting    2.41 0.00        0.53        0.00   
positive someone    as_inexperienced        2.33 0.00        0.87        0.00   
         all        extremely_innovative    5.23 0.00     

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='remotely', axis=0), 'dP1', 15)

dP1      quant    f  unexp_f  LRC  \
polarity l1         l2                                                        
negative nothing    remotely_comparable  0.84  not_exist   42    39.36 3.26   
                    remotely_new         0.72  not_exist   17    15.77 1.66   
         none       remotely_accurate    0.67  not_exist   11    10.72 2.51   
                    remotely_true        0.65  not_exist   44    42.84 4.66   
         nothing    remotely_funny       0.58  not_exist   32    29.20 2.48   
                    remotely_similar     0.55  not_exist   64    58.06 3.05   
                    remotely_interesting 0.48  not_exist   42    37.57 2.41   
                    remotely_close       0.47  not_exist  139   124.25 3.22   
         none       remotely_possible    0.44  not_exist   23    22.12 3.24   
                    remotely_interesting 0.16  not_exist   14    12.61 1.17   
                    remotely_close       0.11  not_exist   34    29.37 1.54   
positive everyone   remotely_interested  0.20  universal   16    14.68 1.61   
         everything remotely_related     0.18  universal   15    13.32 1.08   

                                          dP2  dP1_simple  dP2_simple     G2  \
polarity l1         l2                                                         
negative nothing    remotely_comparable  0.00        0.89        0.00 210.75   
                    remotely_new         0.00        0.77        0.00  74.96   
         none       remotely_accurate    0.00        0.69        0.00  69.17   
                    remotely_true        0.00        0.67        0.00 272.29   
         nothing    remotely_funny       0.00        0.64        0.00 121.14   
                    remotely_similar     0.00        0.60        0.00 231.32   
                    remotely_interesting 0.00        0.53        0.00 137.10   
                    remotely_close       0.00        0.53        0.00 451.70   
         none       remotely_possible    0.00        0.46        0.00 117.80   
                    remotely_interesting 0.00        0.18        0.00  41.61   
                    remotely_close       0.00        0.13        0.00  80.33   
positive everyone   remotely_interested  0.00        0.22        0.00  53.75   
         everything remotely_related     0.00        0.20        0.00  41.59   

                                                N     f1   f2  exp_f  
polarity l1         l2                                                
negative nothing    remotely_comparable   1761853  98788   47   2.64  
                    remotely_new          1761853  98788   22   1.23  
         none       remotely_accurate     1761853  31021   16   0.28  
                    remotely_true         1761853  31021   66   1.16  
         nothing    remotely_funny        1761853  98788   50   2.80  
                    remotely_similar      1761853  98788  106   5.94  
                    remotely_interesting  1761853  98788   79   4.43  
                    remotely_close        1761853  98788  263  14.75  
         none       remotely_possible     1761853  31021   50   0.88  
                    remotely_interesting  1761853  31021   79   1.39  
                    remotely_close        1761853  31021  263   4.63  
positive everyone   remotely_interested   1761853  32327   72   1.32  
         everything remotely_related      1761853  39516   75   1.68

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='quite', axis=0), 'dP1', 15)

dP1             quant     f  unexp_f  \
polarity l1        l2                                                          
negative never     quite_able          0.85         not_exist   319   297.27   
                   quite_enough        0.73         not_exist   150   138.17   
                   quite_satisfied     0.69         not_exist   144   132.11   
                   quite_sure          0.67         not_exist  2176 1,990.17   
                   quite_clear         0.64         not_exist   399   363.75   
                   quite_complete      0.63         not_exist    31    28.20   
                   quite_certain       0.57         not_exist   117   105.48   
                   quite_free          0.56         not_exist    18    16.19   
                   quite_believable    0.47         not_exist    15    13.26   
                   quite_right         0.46         not_exist   217   191.16   
                   quite_comfortable   0.40         not_exist    67    58.03   
                   quite_perfect       0.38         not_exist    22    18.89   
                   quite_ready         0.34         not_exist    59    49.78   
         nobody    quite_sure          0.22         not_exist   660   649.31   
         none      quite_right         0.20         not_exist    89    81.69   
positive all       quite_innovative    0.80         universal   370   314.54   
         something quite_special       0.75       existential   404   325.58   
                   quite_magical       0.72       existential    95    76.03   
                   quite_extraordinary 0.64       existential   164   128.04   
         often     quite_witty         0.61  hedged_universal    12    11.06   
         something quite_profound      0.61       existential    61    47.09   
         some      quite_graphic       0.59       existential    31    27.93   
                   quite_sophisticated 0.59       existential    32    28.80   
         something quite_unique        0.55       existential   224   168.89   
         some      quite_elaborate     0.54       existential    45    40.17   
         something quite_remarkable    0.54       existential   146   109.32   
                   quite_different     0.54       existential  2268 1,697.03   
         some      quite_obscure       0.53       existential    27    24.06   
         something quite_unusual       0.51       existential    87    64.23   
                   quite_unexpected    0.51       existential   126    92.93   

                                        LRC  dP2  dP1_simple  dP2_simple  \
polarity l1        l2                                                      
negative never     quite_able          5.76 0.00        0.91        0.00   
                   quite_enough        4.36 0.00        0.79        0.00   
                   quite_satisfied     4.15 0.00        0.75        0.00   
                   quite_sure          5.00 0.02        0.73        0.02   
                   quite_clear         4.41 0.00        0.70        0.00   
                   quite_complete      2.42 0.00        0.69        0.00   
                   quite_certain       3.44 0.00        0.63        0.00   
                   quite_free          1.48 0.00        0.62        0.00   
                   quite_believable    1.01 0.00        0.54        0.00   
                   quite_right         3.24 0.00        0.52        0.00   
                   quite_comfortable   2.34 0.00        0.47        0.00   
                   quite_perfect       1.23 0.00        0.44        0.00   
                   quite_ready         1.94 0.00        0.40        0.00   
         nobody    quite_sure          5.96 0.10        0.22        0.10   
         none      quite_right         2.95 0.00        0.21        0.00   
positive all       quite_innovative    4.87 0.00        0.94        0.00   
         something quite_special       4.38 0.00        0.93        0.00   
                   quite_magical   

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='that_', axis=0), 'dP1', 15)

dP1      quant    f  unexp_f  LRC  dP2  \
polarity l1    l2                                                         
negative never that_comfortable 0.85  not_exist   22    20.51 1.31 0.00   
               that_fond        0.85  not_exist   22    20.51 1.31 0.00   
               that_nervous     0.81  not_exist   21    19.51 1.65 0.00   
               that_keen        0.81  not_exist   27    25.07 2.27 0.00   
               that_popular     0.79  not_exist   60    55.64 3.69 0.00   
               that_high        0.78  not_exist   38    35.20 2.98 0.00   
               that_happy       0.76  not_exist   37    34.20 2.92 0.00   
               that_close       0.76  not_exist   55    50.83 3.49 0.00   
               that_low         0.70  not_exist   19    17.44 1.73 0.00   
               that_interested  0.68  not_exist   50    45.83 3.16 0.00   
               that_successful  0.65  not_exist   22    20.07 1.95 0.00   
               that_easy        0.60  not_exist  332   300.86 4.12 0.00   
               that_strong      0.59  not_exist   37    33.45 2.52 0.00   
               that_great       0.57  not_exist  214   192.83 3.76 0.00   
               that_serious     0.57  not_exist   32    28.82 2.29 0.00   
positive all   that_different   0.41  universal   42    31.25 1.01 0.00   

                                 dP1_simple  dP2_simple       G2        N  \
polarity l1    l2                                                           
negative never that_comfortable        0.92        0.00   108.64  1761853   
               that_fond               0.92        0.00   108.64  1761853   
               that_nervous            0.88        0.00    98.90  1761853   
               that_keen               0.87        0.00   126.59  1761853   
               that_popular            0.86        0.00   277.04  1761853   
               that_high               0.84        0.00   173.00  1761853   
               that_happy              0.82        0.00   164.36  1761853   
               that_close              0.82        0.00   243.96  1761853   
               that_low                0.76        0.00    78.71  1761853   
               that_interested         0.75        0.00   203.93  1761853   
               that_successful         0.71        0.00    85.96  1761853   
               that_easy               0.66        0.00 1,227.51  1761853   
               that_strong             0.65        0.00   134.15  1761853   
               that_great              0.63        0.00   756.46  1761853   
               that_serious            0.63        0.00   112.78  1761853   
positive all   that_different          0.55        0.00    70.13  1761853   

                                     f1   f2  exp_f  
polarity l1    l2                                    
negative never that_comfortable  109723   24   1.49  
               that_fond         109723   24   1.49  
               that_nervous      109723   24   1.49  
               that_keen         109723   31   1.93  
               that_popular      109723   70   4.36  
               that_high         109723   45   2.80  
               that_happy        109723   45   2.80  
               that_close        109723   67   4.17  
               that_low          109723   25   1.56  
               that_interested   109723   67   4.17  
               that_successful   109723   31   1.93  
               that_easy         109723  500  31.14  
               that_strong       109723   57   3.55  
               that_great        109723  340  21.17  
               that_serious      109723   51   3.18  
positive all   that_different    249279   76  10.75

In [ ]:
show_metric_top(sig_trig_abbr.filter(like='too_', axis=0), 'dP1', 15)

dP1      quant      f  unexp_f  LRC  dP2  \
polarity l1      l2                                                          
negative none    too_pleased     0.92  not_exist   1272 1,248.12 8.80 0.04   
         nothing too_fancy       0.90  not_exist    521   490.55 6.85 0.01   
                 too_major       0.90  not_exist    187   176.07 5.66 0.00   
                 too_special     0.89  not_exist    450   423.37 6.56 0.00   
         never   too_early       0.89  not_exist   4890 4,568.77 7.59 0.04   
                 too_careful     0.88  not_exist    649   606.28 6.66 0.01   
         nothing too_spectacular 0.88  not_exist    108   101.50 4.82 0.00   
                 too_drastic     0.86  not_exist    103    96.72 4.74 0.00   
         none    too_flattering  0.84  not_exist     18    17.63 3.27 0.00   
         never   too_safe        0.83  not_exist    213   198.12 5.24 0.00   
         none    too_thrilled    0.81  not_exist    115   112.55 6.22 0.00   
         nothing too_exciting    0.80  not_exist    401   374.76 5.57 0.00   
         never   too_late        0.79  not_exist  10770 9,974.22 6.14 0.10   
         nothing too_egregious   0.79  not_exist     16    14.93 1.32 0.00   
                 too_strenuous   0.78  not_exist     93    86.78 4.33 0.00   
positive all     too_commonplace 0.86  universal    124   106.46 1.31 0.00   
                 too_possible    0.85  universal    144   123.48 1.53 0.00   
                 too_plausible   0.85  universal    105    90.00 1.07 0.00   
                 too_common      0.85  universal   4882 4,182.91 7.90 0.02   
                 too_evident     0.85  universal    408   349.57 5.26 0.00   
                 too_familiar    0.84  universal   6933 5,934.39 7.65 0.03   
                 too_aware       0.84  universal   2229 1,908.11 7.11 0.01   
                 too_typical     0.84  universal    284   243.11 4.73 0.00   
                 too_apparent    0.84  universal    556   475.78 5.76 0.00   
                 too_human       0.83  universal    628   536.60 5.76 0.00   
                 too_believable  0.83  universal     97    82.85 2.80 0.00   
                 too_true        0.83  universal    471   402.24 5.42 0.00   
                 too_prevalent   0.83  universal    211   180.16 4.36 0.00   
                 too_real        0.82  universal   2250 1,920.19 6.47 0.01   
                 too_likely      0.82  universal    282   240.40 4.74 0.00   

                                  dP1_simple  dP2_simple        G2        N  \
polarity l1      l2                                                           
negative none    too_pleased            0.94        0.04  9,701.23  1761853   
         nothing too_fancy              0.96        0.01  2,823.11  1761853   
                 too_major              0.96        0.00  1,012.04  1761853   
                 too_special            0.95        0.00  2,401.96  1761853   
         never   too_early              0.95        0.04 25,284.55  1761853   
                 too_careful            0.95        0.01  3,323.75  1761853   
         nothing too_spectacular        0.93        0.00    565.14  1761853   
                 too_drastic            0.92        0.00    532.02  1761853   
         none    too_flattering         0.86        0.00    128.31  1761853   
         never   too_safe               0.89        0.00  1,021.95  1761853   
         none    too_thrilled           0.83        0.00    802.44  1761853   
         nothing too_exciting           0.86        0.00  1,935.54  1761853   
         never   too_late               0.84        0.10 49,945.41  1761853   
         nothing too_egregious          0.84        0.00     75.97  1761853   
                 too_strenuous          0.84        0.00    439.65  1761853   
positive all     too_commonplace        1.00        0.00    485.03  1761853   
                 too_possible           0.99        0.00    551.63  1761853   
                 too_plausible      